In [1]:
%matplotlib inline

import librosa
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import torch
import os
import glob
import pickle
import copy
import random
import time
import traceback
from multiprocessing import Process

# Make file index dict

In [4]:
OPT_INDEX = '/Lun0/zhiyong/vox_small_noVAD'
# data_dir = '/Lun0/zhiyong/dataset/vox2/dev/aac'
data_dir_1 = '/Lun0/zhiyong/dataset/vox1/dev/wav'
# data_dir_2test = '/Lun0/zhiyong/dataset/vox2/test/aac'


spk2id = os.path.join(OPT_INDEX, 'spk2id')
spk2utt_dict = os.path.join(OPT_INDEX, 'spk2utt_dict')
spk2utt_train_dict = os.path.join(OPT_INDEX, 'spk2utt_train_dict')
spk2utt_val_dict = os.path.join(OPT_INDEX, 'spk2utt_val_dict')
val_utt_num = 200

In [5]:
spk2utt = {}
spk2utt_train = {}
spk2utt_val = {}

with open(spk2id, 'w') as f:
    pass

# label_dir = glob.glob(data_dir+'/*')
# label = [i.split('/')[-1] for i in label_dir]

label_dir = glob.glob(data_dir_1+'/*')
label_1 = [i.split('/')[-1] for i in label_dir]

# label_dir = glob.glob(data_dir_2test+'/*')
# label_2test = [i.split('/')[-1] for i in label_dir]

# for count, i in enumerate(label):
#     with open(spk2id, 'a') as f:
#         f.write(str(count)+','+i+'\n')
#     spk2utt[count] = glob.glob(os.path.join(data_dir, i)+'/*/*.wav')

for count_1, i in enumerate(label_1):
    with open(spk2id, 'a') as f:
        f.write(str(count_1)+','+i+'\n')
    spk2utt[count_1] = glob.glob(os.path.join(data_dir_1, i)+'/*/*.wav')

# for count_2test, i in enumerate(label_2test):
#     with open(spk2id, 'a') as f:
#         f.write(str(count+1+count_1+1+count_2test)+','+i+'\n')
#     spk2utt[count+1+count_1+1+count_2test] = glob.glob(os.path.join(data_dir_2test, i)+'/*/*.wav')

In [6]:
len(spk2utt)

1211

In [7]:
count = 0
count_list = []
for i in spk2utt:
    count += len(spk2utt[i])
    count_list.append(len(spk2utt[i]))
print(count)

148642


In [8]:
with open(spk2utt_dict, 'wb') as handle:
    pickle.dump(spk2utt, handle)

with open(spk2utt_dict, 'rb') as handle:
    spk2utt = pickle.load(handle)

spk2utt_train = copy.deepcopy(spk2utt)
for i in range(val_utt_num):
    spk2utt_val[i] = [spk2utt[i][0]]
    spk2utt_train[i] = spk2utt[i][1:]

In [9]:
count = 0
count_list = []
for i in spk2utt_train:
    count += len(spk2utt_train[i])
    count_list.append(len(spk2utt[i]))
print(count)

148442


In [10]:
with open(spk2utt_train_dict, 'wb') as handle:
    pickle.dump(spk2utt_train, handle)

In [11]:
count = 0
count_list = []
for i in spk2utt_val:
    count += len(spk2utt_val[i])
    count_list.append(len(spk2utt[i]))
print(count)

200


In [12]:
with open(spk2utt_val_dict, 'wb') as handle:
    pickle.dump(spk2utt_val, handle)

In [13]:
music_dir = '/Lun0/zhiyong/dataset/musan/music'
noise_dir = '/Lun0/zhiyong/dataset/musan/noise'
speech_dir = '/Lun0/zhiyong/dataset/musan/speech'
rir_dir = '/Lun0/zhiyong/dataset/RIRS_NOISES'

music_dict_out = os.path.join(OPT_INDEX, 'music_dict')
noise_dict_out = os.path.join(OPT_INDEX, 'noise_dict')
babble_dict_out = os.path.join(OPT_INDEX, 'babble_dict')
rir_dict_out = os.path.join(OPT_INDEX, 'rir_dict')

In [14]:
music_dict = {}
noise_dict = {}
babble_dict = {}
rir_dict = {}

In [15]:
tmp_dir_list = glob.glob(music_dir+'/*/*.wav')
for count, this_dir in enumerate(tmp_dir_list):
    music_dict[count] = this_dir
with open(music_dict_out, 'wb') as handle:
    pickle.dump(music_dict, handle)

In [16]:
tmp_dir_list = glob.glob(noise_dir+'/*/*.wav')
for count, this_dir in enumerate(tmp_dir_list):
    noise_dict[count] = this_dir
with open(noise_dict_out, 'wb') as handle:
    pickle.dump(noise_dict, handle)

In [17]:
tmp_dir_list = glob.glob(speech_dir+'/*/*.wav')
for count, this_dir in enumerate(tmp_dir_list):
    babble_dict[count] = this_dir
with open(babble_dict_out, 'wb') as handle:
    pickle.dump(babble_dict, handle)

In [18]:
tmp_dir_list = glob.glob(rir_dir+'/simulated_rirs/smallroom/*/*.wav')
for count, this_dir in enumerate(tmp_dir_list):
    rir_dict[count] = this_dir
tmp_dir_list = glob.glob(rir_dir+'/simulated_rirs/mediumroom/*/*.wav')
for count_2, this_dir in enumerate(tmp_dir_list):
    rir_dict[count +1 + count_2] = this_dir

with open(rir_dict_out, 'wb') as handle:
    pickle.dump(rir_dict, handle)

In [13]:
music_len = os.path.join(OPT_INDEX, 'music_len')
noise_len = os.path.join(OPT_INDEX, 'noise_len')
babble_len = os.path.join(OPT_INDEX, 'babble_len')
spk2utt_train_len = os.path.join(OPT_INDEX, 'spk2utt_train_len')
spk2utt_val_len = os.path.join(OPT_INDEX, 'spk2utt_val_len')

music_len_dict = {}
noise_len_dict = {}
babble_len_dict = {}
spk2utt_train_len_dict = {}
spk2utt_val_len_dict = {}

In [20]:
with open(music_dict_out, 'rb') as handle:
    music_dict = pickle.load(handle)

for i in music_dict:
    music_len_dict[i] = librosa.core.load(music_dict[i], sr=16000)[0].shape[0]

with open(music_len, 'wb') as handle:
    pickle.dump(music_len_dict, handle)

In [21]:
with open(noise_dict_out, 'rb') as handle:
    noise_dict = pickle.load(handle)

for i in noise_dict:
    noise_len_dict[i] = librosa.core.load(noise_dict[i], sr=16000)[0].shape[0]

with open(noise_len, 'wb') as handle:
    pickle.dump(noise_len_dict, handle)

In [22]:
with open(babble_dict_out, 'rb') as handle:
    babble_dict = pickle.load(handle)

for i in babble_dict:
    babble_len_dict[i] = librosa.core.load(babble_dict[i], sr=16000)[0].shape[0]

with open(babble_len, 'wb') as handle:
    pickle.dump(babble_len_dict, handle)

In [14]:
with open(spk2utt_train_dict, 'rb') as handle:
    spk2utt_train = pickle.load(handle)

for i in spk2utt_train:
    for utt in spk2utt_train[i]:
        spk2utt_train_len_dict[utt] = librosa.core.load(utt, sr=16000)[0].shape[0]

with open(spk2utt_train_len, 'wb') as handle:
    pickle.dump(spk2utt_train_len_dict, handle)

In [15]:
with open(spk2utt_val_dict, 'rb') as handle:
    spk2utt_val = pickle.load(handle)

for i in spk2utt_val:
    for utt in spk2utt_val[i]:
        spk2utt_val_len_dict[utt] = librosa.core.load(utt, sr=16000)[0].shape[0]

with open(spk2utt_val_len, 'wb') as handle:
    pickle.dump(spk2utt_val_len_dict, handle)

In [16]:
# trial dict
trial_dir = '/Lun0/zhiyong/dataset/vox1/test/wav'
trial_dict_out = os.path.join(OPT_INDEX, 'trial_dict')
trial_dict = {}

tmp_dir_list = glob.glob(trial_dir+'/*/*/*.wav')
for count, this_dir in enumerate(tmp_dir_list):
    trial_dict[count] = this_dir
with open(trial_dict_out, 'wb') as handle:
    pickle.dump(trial_dict, handle)

In [17]:
len(trial_dict)

4874

# Iterable Dataset & Make train data

In [18]:
fft = librosa.get_fftlib()
class VoxIterableDataset(object):
    def __init__(self, data_dir_dict, data_len_dict, config):        
        with open(data_dir_dict['spk2utt_train_dict'], 'rb') as handle:
            self.spk2utt_train_dict = pickle.load(handle)
        with open(data_dir_dict['music_dict'], 'rb') as handle:
            self.music_dict = pickle.load(handle)
        with open(data_dir_dict['noise_dict'], 'rb') as handle:
            self.noise_dict = pickle.load(handle)
        with open(data_dir_dict['babble_dict'], 'rb') as handle:
            self.babble_dict = pickle.load(handle)
        with open(data_dir_dict['rir_dict'], 'rb') as handle:
            self.rir_dict = pickle.load(handle)
            
        with open(data_len_dict['spk2utt_train_len'], 'rb') as handle:
            self.spk2utt_train_len = pickle.load(handle)
        with open(data_len_dict['music_len'], 'rb') as handle:
            self.music_len = pickle.load(handle)
        with open(data_len_dict['noise_len'], 'rb') as handle:
            self.noise_len = pickle.load(handle)
        with open(data_len_dict['babble_len'], 'rb') as handle:
            self.babble_len = pickle.load(handle)
        
        
        self.random_spkrs_batchlist = None
        self.ramdom_batch_len = None
        self.random_noise_type = None
        
        
        self.possible_babble_num = [3, 4, 5, 6, 7]
        self.possible_babble_snr = [13, 15, 17, 20]
        self.possible_noise_snr = [0, 5, 10, 15]
        self.possible_music_snr = [5, 8, 10, 15]
        
        self.sr = config['sr']
        self.repeats = config['repeats']
        self.batch_size = config['batch_size']
        self.extended_prefectch = config['extended_prefectch']
        
        self.mfcc_dim = 30
        
        # Auxiliary paras
        self.multi_read_count = 0
        self.preload_mem = False
        
        
        
    def __iter__(self):
        return self
    
    def __next__(self):
        assert len(self.ramdom_batch_len) == len(self.random_spkrs_batchlist)
        try:
            batch_frame_len = self.ramdom_batch_len.pop(0)
            batch_spkrs = self.random_spkrs_batchlist.pop(0)
            batch_noise_type = self.random_noise_type.pop(0)
            batched_feats = np.zeros([self.batch_size, batch_frame_len, self.mfcc_dim])
            batched_labels = np.zeros(self.batch_size)
            
            for batch_index, (spkr, noise_type) in enumerate(zip(batch_spkrs, batch_noise_type)):
                
                concat_wav, VAD_result = self._colleting_and_slicing(spkr, batch_frame_len,\
                hop_len=160, extended_prefectch=self.extended_prefectch)
            
                
                if noise_type == 0:
                    aug_wav = concat_wav
                
                elif noise_type == 1:
                    aug_wav = self._add_rebverb(concat_wav)
                   
                elif noise_type == 2:
                    aug_wav = self._add_noise(concat_wav)
                    
                elif noise_type == 3:
                    aug_wav = self._add_music(concat_wav)
                  
                elif noise_type == 4:
                    aug_wav = self._add_babble(concat_wav)
             
                else:
                    raise NotImplementedError
                    
            
                single_feats = librosa.feature.mfcc(y=aug_wav, sr=self.sr, n_mfcc=30, \
                dct_type=2, n_fft=512, hop_length=160, \
                win_length=None, window='hann', power=2.0, \
                center=True, pad_mode='reflect', n_mels=30, \
                fmin=20, fmax=7600)
                # Note single_feats needs transpose
                out_feats = self._CMVN(single_feats.T, cmn_window = 300, normalize_variance = False)
                # Apply VAD
                assert out_feats.shape[0] == VAD_result.shape[0]
#                 out_feats = out_feats[VAD_result.astype(np.bool)]
                batched_feats[batch_index] = out_feats[:batch_frame_len]
                batched_labels[batch_index] = spkr
                
            return batched_feats, batched_labels
        
        except IndexError:
            raise StopIteration

    def process_one_utt(self, utt_dir):
        try:
            concat_wav, _ = librosa.load(utt_dir, sr=self.sr)
            
            VAD_result = self._VAD_detection(concat_wav)
            
            aug_wav = concat_wav

            single_feats = librosa.feature.mfcc(y=aug_wav, sr=self.sr, n_mfcc=30, \
            dct_type=2, n_fft=512, hop_length=160, \
            win_length=None, window='hann', power=2.0, \
            center=True, pad_mode='reflect', n_mels=30, \
            fmin=20, fmax=7600)
            # Note single_feats needs transpose
            out_feats = self._CMVN(single_feats.T, cmn_window = 300, normalize_variance = False)
            # Apply VAD
            assert out_feats.shape[0] == VAD_result.shape[0]
            out_feats = out_feats[VAD_result.astype(np.bool)]
            
            batched_feats = out_feats[None, :, :]
                
            return batched_feats
        
        except Exception:
            traceback.print_exc()

    def process_one_utt_noVAD(self, utt_dir):
        try:
            concat_wav, _ = librosa.load(utt_dir, sr=self.sr)
            
            aug_wav = concat_wav

            single_feats = librosa.feature.mfcc(y=aug_wav, sr=self.sr, n_mfcc=30, \
            dct_type=2, n_fft=512, hop_length=160, \
            win_length=None, window='hann', power=2.0, \
            center=True, pad_mode='reflect', n_mels=30, \
            fmin=20, fmax=7600)
            # Note single_feats needs transpose
            out_feats = self._CMVN(single_feats.T, cmn_window = 300, normalize_variance = False)
            
            batched_feats = out_feats[None, :, :]
                
            return batched_feats
        
        except Exception:
            traceback.print_exc()            
    
    def noise_data_preload(self):
        print('preloading music_dict')
        for count, i in enumerate(self.music_dict):
            _, _ = librosa.load(self.music_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading noise_dict')        
        for count, i in enumerate(self.noise_dict):
            _, _ = librosa.load(self.noise_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading babble_dict')        
        for count, i in enumerate(self.babble_dict):
            _, _ = librosa.load(self.babble_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
    
    def noise_data_preload2mem(self):
        print('preloading to memory')
        
        self.music_preload_dict = {}
        self.noise_preload_dict = {}
        self.babble_preload_dict = {}
        self.preload_mem = True
        print('preloading music_dict')
        for count, i in enumerate(self.music_dict):
            self.music_preload_dict[i], _ = librosa.load(self.music_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading noise_dict')        
        for count, i in enumerate(self.noise_dict):
            self.noise_preload_dict[i], _ = librosa.load(self.noise_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading babble_dict')        
        for count, i in enumerate(self.babble_dict):
            self.babble_preload_dict[i], _ = librosa.load(self.babble_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)       
        
        
    def get_random_list(self):
        spkrs_list = self.repeats * list(self.spk2utt_train_dict.keys())
        random.shuffle(spkrs_list)
        len_spkrs_list = len(spkrs_list)
        self.random_spkrs_batchlist = [spkrs_list[i*self.batch_size:i*self.batch_size+self.batch_size]\
        for i in range(len_spkrs_list // self.batch_size)]
        
        self.ramdom_batch_len = [random.randint(200, 400) for i in range(len_spkrs_list // self.batch_size)]
        
        noise_type_list = [i%5 for i in range(len_spkrs_list)]

        random.shuffle(noise_type_list)
        self.random_noise_type = [noise_type_list[i*self.batch_size:i*self.batch_size+self.batch_size]\
        for i in range(len_spkrs_list // self.batch_size)]
        
        assert len(self.random_spkrs_batchlist) == len(self.ramdom_batch_len)\
        == len(self.random_noise_type)
        
    def _colleting_and_slicing(self, spkr, batch_frame_len, hop_len=160, extended_prefectch=2.0):
        
        least_wav_len = (batch_frame_len - 1) * hop_len
        concat_utt = np.zeros(0)
        valid_frames_len = 0
        
        # Use to count multi_read_count
        get_count = 0

        while valid_frames_len < batch_frame_len:
            concat_utt = np.zeros(0)

            utt_dir = self._get_random_spk_utt(spkr, self.spk2utt_train_dict)
            utt_len = self.spk2utt_train_len[utt_dir]
#             off = self._get_random_offset(least_wav_len, utt_len) / self.sr
            off = self._get_random_offset(least_wav_len+extended_prefectch*self.sr, utt_len) / self.sr
            dur = least_wav_len / self.sr + extended_prefectch
            
            utt_part, _ = librosa.load(utt_dir, sr=self.sr, offset=off, duration=dur)
            
            concat_utt = np.append(concat_utt, utt_part)
            detected_frames = self._VAD_detection(concat_utt)
            valid_frames_len = np.sum(detected_frames)

            get_count += 1

        if get_count > 1:
            self.multi_read_count += 1

        VAD_result = detected_frames
        return concat_utt, VAD_result
    
    def _add_rebverb(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = in_wav
        filter_dir = self._get_random_noise(self.rir_dict)
        filter, _ = librosa.load(filter_dir, sr=self.sr)
        
        signal_length = len(signal)
        filter_length = len(filter)
        output_length = signal_length + filter_length - 1
        output = np.zeros(output_length)

        fft_length = 2**np.ceil(np.log2(4 * filter_length)).astype(np.int)
        block_length = fft_length - filter_length + 1


        filter_padded = np.zeros(fft_length)
        filter_padded[0:filter_length] = filter
        filter_padded = fft.rfft(filter_padded)



        for i in range(signal_length//block_length + 1):
            process_length = min(block_length, signal_length - i * block_length);
            signal_block_padded = np.zeros(fft_length)
            signal_block_padded[0:process_length] = signal[i * block_length : i * block_length + process_length]
            signal_block_padded = fft.rfft(signal_block_padded)

            signal_block_padded = filter_padded * signal_block_padded

            signal_block_padded = fft.irfft(signal_block_padded, n=fft_length)

            if (i*block_length + fft_length) <= output_length:
                output[i*block_length : i*block_length + fft_length] += signal_block_padded
            else:
                output[i*block_length : output_length] += signal_block_padded[:output_length-i*block_length]
        
        # shift with max index of filter
        shift_index = np.argmax(filter)
        
        final_out = output[shift_index:shift_index+signal_length]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_noise(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]
        
        signal_len = len(signal)
        total_noise_len = 0
        signal_off = 0
        while total_noise_len < signal_len:
            
            noise_dir, noise_index = self._get_random_noise(self.noise_dict, return_index=True)
            noise_len = self.noise_len[noise_index]
            if noise_len > signal_len:
                noise_off = self._get_random_offset(signal_len, noise_len)
                total_noise_len += signal_len
                if self.preload_mem:
                    noise = self.noise_preload_dict[noise_index][noise_off:noise_off+signal_len]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr, offset=noise_off/self.sr,\
                    duration=signal_len/self.sr)
                
            else:
                total_noise_len += noise_len
                if self.preload_mem:
                    noise = self.noise_preload_dict[noise_index]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr)
                
            snr_db = self.possible_noise_snr[random.randint(0, len(self.possible_noise_snr)-1)]
        
            signal = self._add_db(signal, noise, signal_off, snr_db, power_before_reverb)
            
            signal_off += len(noise)
        
        output = signal
        final_out = output[shift_index:shift_index+signal_len]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_music(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]
        
        signal_len = len(signal)
        total_noise_len = 0
        signal_off = 0
        while total_noise_len < signal_len:
            
            noise_dir, noise_index = self._get_random_noise(self.music_dict, return_index=True)
            noise_len = self.music_len[noise_index]
            if noise_len > signal_len:
                noise_off = self._get_random_offset(signal_len, noise_len)
                total_noise_len += signal_len
                if self.preload_mem:
                    noise = self.music_preload_dict[noise_index][noise_off:noise_off+signal_len]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr, offset=noise_off/self.sr,\
                    duration=signal_len/self.sr)
            else:
                total_noise_len += noise_len
                if self.preload_mem:
                    noise = self.music_preload_dict[noise_index]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr)
                
            snr_db = self.possible_music_snr[random.randint(0, len(self.possible_music_snr)-1)]
        
            signal = self._add_db(signal, noise, signal_off, snr_db, power_before_reverb)
            
            signal_off += len(noise)
        
        output = signal
        final_out = output[shift_index:shift_index+signal_len]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_babble(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]
        
        signal_len = len(signal)
        signal_off = 0
        bg_spks_num = self.possible_babble_num[random.randint(0, len(self.possible_babble_num)-1)]    
        for _ in range(bg_spks_num):            
            noise_dir, noise_index = self._get_random_noise(self.babble_dict, return_index=True)
            noise_len = self.babble_len[noise_index]
            if noise_len > signal_len:
                noise_off = self._get_random_offset(signal_len, noise_len)
                if self.preload_mem:
                    noise = self.babble_preload_dict[noise_index][noise_off:noise_off+signal_len]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr, offset=noise_off/self.sr,\
                    duration=signal_len/self.sr)
            else:
                if self.preload_mem:
                    noise = self.babble_preload_dict[noise_index]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr)
                
            snr_db = self.possible_babble_snr[random.randint(0, len(self.possible_babble_snr)-1)]
        
            signal = self._add_db(signal, noise, signal_off, snr_db, power_before_reverb)
            
        output = signal
        final_out = output[shift_index:shift_index+signal_len]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_db(self, in_wav, noise, signal_off, snr_db, power_before_reverb):
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]

        noise_power = noise.dot(noise) / len(noise)
        scale_factor = np.sqrt(10**(-snr_db / 10) * power_before_reverb / noise_power)
        noise = scale_factor * noise

        add_length = min(len(noise), len(signal)-signal_off)
        signal[signal_off:signal_off+add_length] += noise[:add_length]
        out_wav = signal      
        
        return out_wav
    
    def _CMVN(self, in_feat, cmn_window = 300, normalize_variance = False):             
        num_frames = in_feat.shape[0]
        dim = in_feat.shape[1]
        last_window_start = -1
        last_window_end = -1
        cur_sum = np.zeros(dim)
        cur_sumsq = np.zeros(dim)

        out_feat = np.zeros([num_frames, dim])

        for t in range(num_frames):
            window_start = 0
            window_end = 0

            window_start = t - int(cmn_window / 2)
            window_end = window_start + cmn_window

            if (window_start < 0):
                window_end -= window_start
                window_start = 0

            if (window_end > num_frames):
                window_start -= (window_end - num_frames)
                window_end = num_frames
                if (window_start < 0):
                    window_start = 0

            if (last_window_start == -1):
                input_part = in_feat[window_start:window_end]
                cur_sum = np.sum(input_part, axis=0, keepdims=False)
                if normalize_variance:
                    cur_sumsq = np.sum(input_part**2, axis=0, keepdims=False)
            else:
                if (window_start > last_window_start):
                    frame_to_remove = in_feat[last_window_start]
                    cur_sum -= frame_to_remove
                    if normalize_variance:
                        cur_sumsq -= frame_to_remove**2

                if (window_end > last_window_end):
                    frame_to_add = in_feat[last_window_end]
                    cur_sum += frame_to_add
                    if normalize_variance:
                        cur_sumsq += frame_to_add**2

            window_frames = window_end - window_start
            last_window_start = window_start
            last_window_end = window_end

            out_feat[t] = in_feat[t] - (1.0 / window_frames) * cur_sum


            if normalize_variance:
                if (window_frames == 1):
                    out_feat[t] = 0.0
                else:
                    variance = (1.0 / window_frames) * cur_sumsq - (1.0 / window_frames**2) * cur_sum**2
                    variance = np.maximum(1.0e-10, variance)
                    out_feat[t] /= variance**(0.5)
                    
        return out_feat

    def _get_random_noise(self, noise_dict, return_index=False):
        dict_len = len(noise_dict)
        i = random.randint(0, dict_len-1)
        noise_dir = noise_dict[i]
        
        if return_index:
            return noise_dir, i
        else:
            return noise_dir
    
    def _get_random_spk_utt(self, spkr, spk2utt):
        this_utts = spk2utt[spkr]
        this_num_utts = len(this_utts)
        i = random.randint(0, this_num_utts-1)
        utt_dir = this_utts[i]
        return utt_dir

    def _get_random_offset(self, expected_length, utt_len):
        if expected_length > utt_len:
            return 0
        
        free_length = utt_len - expected_length
        offset = random.randint(0, free_length)
        return offset
        
    @property
    def _VAD_config(self):
        vad_energy_threshold = -3.0
        vad_energy_mean_scale = 1.0
        vad_frames_context = 0
        vad_proportion_threshold = 0.12
        
        return vad_energy_threshold, vad_energy_mean_scale,\
        vad_frames_context, vad_proportion_threshold
        
        
    def _VAD_detection(self, wav):
        vad_energy_threshold, vad_energy_mean_scale,\
        vad_frames_context, vad_proportion_threshold = self._VAD_config
        
        y_tmp = np.pad(wav, int(512 // 2), mode='reflect')
        y_tmp = librosa.util.frame(y_tmp, frame_length=512, hop_length=160)
        y_log_energy = np.log(np.maximum(np.sum(y_tmp**2, axis=0), 1e-15))

        T = len(y_log_energy)
        output_voiced = np.zeros(T)
        if (T == 0):
            raise Exception("zero wave length")

        energy_threshold = vad_energy_threshold
        if (vad_energy_mean_scale != 0.0):
            assert(vad_energy_mean_scale > 0.0)
            energy_threshold += vad_energy_mean_scale * np.sum(y_log_energy) / T


        assert(vad_frames_context >= 0)
        assert(vad_proportion_threshold > 0.0 and vad_proportion_threshold < 1.0);

        for t in range(T):
            num_count = 0
            den_count = 0
            context = vad_frames_context
            for t2 in range(t - context, t + context+1):
                if (t2 >= 0 and t2 < T):
                    den_count+=1
                    if (y_log_energy[t2] > energy_threshold):
                        num_count+=1

            if (num_count >= den_count * vad_proportion_threshold):
                output_voiced[t] = 1.0
            else:
                output_voiced[t] = 0.0
        
        return output_voiced

In [19]:
OPT_INDEX_1 = '/Lun0/zhiyong/dataset'
data_dir_dict = {}

# train
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_train_dict')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX_1, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX_1, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX_1, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX_1, 'rir_dict')

In [20]:
data_len_dict = {}

data_len_dict['music_len'] = os.path.join(OPT_INDEX_1, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX_1, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX_1, 'babble_len')
data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX, 'spk2utt_train_len')
data_len_dict['spk2utt_val_len'] = os.path.join(OPT_INDEX, 'spk2utt_val_len')

In [21]:
config = {}

config['sr'] = 16000
config['repeats'] = 100
config['batch_size'] = 128
config['extended_prefectch'] = 1.0

def iter_data_preload(dataset, i):
    dataset.get_random_list()
    try:
        start_time = time.time()
        for count, (data, label) in enumerate(dataset):
            with open('/Lun0/zhiyong/vox_small_noVAD/train_data/'+str(i)+str('_')+str(count), 'wb') as handle:
                pickle.dump((data.astype(np.float16), label.astype(np.int16)), handle)
            print(time.time()-start_time)
            with open('/Lun0/zhiyong/vox_small_noVAD/tmp_data_errlog', 'a') as f:
                f.write(str(time.time()-start_time)+'\n')
            start_time = time.time()
    except Exception:
        with open('/Lun0/zhiyong/vox_small_noVAD/tmp_data_errlog', 'a') as f:
            traceback.print_exc(file=f)
        raise Exception

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)
# dataset.noise_data_preload2mem()
dataset.noise_data_preload()

processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(2)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

preloading music_dict
100
200
300
400
500
600
preloading noise_dict
100
200
300
400
500
600
700
800
900
preloading babble_dict
100
200
300
400
2.649683952331543
3.4718143939971924
3.28725528717041
2.7665932178497314
2.439974784851074
3.0702531337738037
3.2865450382232666
3.0985052585601807
2.7716476917266846
3.5571162700653076
2.6205434799194336
2.961494207382202
3.4678761959075928
3.4969141483306885
3.0565996170043945
3.2262816429138184
2.926072597503662
3.6643826961517334
2.4876110553741455
3.2256147861480713
3.3747270107269287
2.7149956226348877
2.6093251705169678
3.187929630279541
2.662672281265259
2.752714157104492
3.181025505065918
3.135780096054077
3.0168516635894775
3.0784060955047607
3.0359392166137695
3.5979974269866943
2.7452986240386963
2.3325085639953613
2.8792853355407715
3.2927486896514893
3.046846628189087
2.6801464557647705
2.7728874683380127
3.302272319793701
2.739293336868286
3.431173086166382
2.7980756759643555
3.2020103931427
3.2599775791168213
2.8715627193450928
2

3.4358177185058594
2.599032163619995
2.3731298446655273
2.7125027179718018
2.6168322563171387
3.1591145992279053
3.4456470012664795
2.801004409790039
2.4819836616516113
2.2069733142852783
3.3446547985076904
2.6675186157226562
2.563375949859619
2.8334925174713135
2.6745896339416504
2.569248676300049
2.2164435386657715
2.575897455215454
2.9551079273223877
2.494535207748413
2.832854986190796
2.983278512954712
2.632732629776001
3.291168451309204
3.070788621902466
3.002924680709839
2.71268892288208
2.4270567893981934
3.2903542518615723
2.2622673511505127
2.4021623134613037
3.4192020893096924
3.439272165298462
2.1001651287078857
3.0206453800201416
2.675161838531494
2.525085926055908
2.565258026123047
3.1299092769622803
3.1993513107299805
2.2173635959625244
2.8114874362945557
2.9836323261260986
3.014346122741699
2.3173646926879883
3.309757709503174
2.175398349761963
3.099224328994751
2.782148599624634
2.4805285930633545
3.225925922393799
2.4959311485290527
3.2269225120544434
3.026498794555664

2.40954852104187
2.844491481781006
2.6521594524383545
2.7813398838043213
2.6916868686676025
3.0393784046173096
2.672365427017212
2.6602935791015625
2.3772361278533936
2.5723624229431152
2.6388704776763916
3.265014410018921
3.2059524059295654
2.827897071838379
2.978482246398926
2.3744544982910156
2.2035000324249268
2.8732943534851074
3.384002208709717
3.3349411487579346
2.7152340412139893
2.725327491760254
3.0586798191070557
3.0593762397766113
3.125678777694702
3.3910746574401855
2.7402353286743164
2.769883155822754
2.7913360595703125
2.66159987449646
2.5390586853027344
2.5825812816619873
2.7801411151885986
2.5125746726989746
3.0668013095855713
2.863629102706909
3.1102609634399414
2.713331699371338
3.614254951477051
2.9891867637634277
3.0659940242767334
2.640584707260132
3.4450302124023438
2.5215766429901123
3.3257594108581543
2.950180768966675
2.995577096939087
2.556211471557617
3.1715087890625
3.1073849201202393
2.809530019760132
2.219043493270874
2.821953535079956
3.208054780960083
3

3.102125883102417
3.2586984634399414
2.9110054969787598
2.819171667098999
3.362621307373047
2.715949058532715
2.3114187717437744
3.161917209625244
2.781675100326538
3.1310648918151855
2.98384952545166
3.1420605182647705
3.187371015548706
3.0722618103027344
3.612576723098755
2.3465309143066406
2.708512544631958
2.8234050273895264
2.6588985919952393
2.4362714290618896
2.5922601222991943
2.4326961040496826
3.3316686153411865
2.5380914211273193
3.1059422492980957
2.539724588394165
3.446944236755371
3.0967581272125244
2.730631113052368
1.989243984222412
2.7194316387176514
2.591135263442993
2.59687876701355
2.7427899837493896
2.3083584308624268
2.817793607711792
2.53678297996521
3.3677680492401123
2.4967195987701416
2.649693012237549
2.893885612487793
2.1478164196014404
2.676265001296997
2.383049964904785
2.305391311645508
2.1584720611572266
2.6012051105499268
1.9850239753723145
2.91369366645813
2.6866912841796875
2.7626049518585205
2.3461556434631348
2.9201571941375732
2.366269111633301
2.9

2.8393170833587646
2.5195467472076416
3.380601167678833
2.660322666168213
3.236799478530884
2.373607873916626
3.305933713912964
2.385619640350342
3.304274082183838
2.6519789695739746
2.939728021621704
3.2890779972076416
2.7850708961486816
3.5038702487945557
2.8073737621307373
2.6041393280029297
2.449100971221924
2.9052770137786865
2.3856730461120605
2.5416297912597656
2.821537733078003
2.81661319732666
2.395110845565796
2.8140852451324463
3.0135486125946045
2.5550806522369385
3.1774556636810303
3.0173285007476807
2.4777002334594727
2.9516468048095703
2.9176597595214844
2.7272660732269287
2.7886531352996826
3.6427431106567383
2.6477560997009277
2.7950072288513184
2.460679769515991
3.0978314876556396
3.179201364517212
2.5394108295440674
3.2091805934906006
3.26242733001709
2.351496458053589
3.373906373977661
3.064504384994507
2.8097763061523438
2.735732078552246
2.924445390701294
2.8667280673980713
2.4018819332122803
2.8795628547668457
2.7494466304779053
2.838157892227173
2.57705092430114

# Make train index

In [22]:
# OPT_INDEX = '/Lun0/zhiyong/dataset'
train_data_dir = os.path.join(OPT_INDEX, 'train_data')
expected_len = 1892
workers = 2
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'vox_small_noVAD_csv.csv')

In [23]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [24]:
count = 0
with open(output, 'w') as f:
    for i in range(workers):
        for j in range(single_worker_len):
            path = os.path.join(train_data_dir, str(i)+'_'+str(j))
            assert os.path.isfile(path)
            f.write(path+'\n')
            count += 1

In [25]:
count

1892

In [108]:
# OPT_INDEX = '/Lun0/zhiyong/dataset'
# train_data_dir = os.path.join(OPT_INDEX, 'tmp_data_2')
# expected_len = 85810
# workers = 10
# single_worker_len = int(expected_len / workers)
# output = os.path.join(OPT_INDEX, 'tmp_data_2_csv.csv')

In [109]:
# assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [110]:
# count = 0
# with open(output, 'w') as f:
#     for i in range(workers):
#         for j in range(single_worker_len):
#             path = os.path.join(train_data_dir, str(i)+'_'+str(j))
#             assert os.path.isfile(path)
#             f.write(path+'\n')
#             count += 1

In [111]:
# count

85810

# Test

In [ ]:
# from torch.utils.data import Dataset, DataLoader

In [ ]:
# class PickleDataSet(Dataset):
#     def __init__(self, spkid2utt_csv):
#         self.spkid2utt = pd.read_csv(spkid2utt_csv, header=None)
        
#     def __getitem__(self, index):
#         path = self.spkid2utt[0][index]
#         with open(path, 'rb') as handle:
#             batch_data = pickle.load(handle)
#         batch_feats = batch_data[0].astype(np.float32)
#         batch_labels = batch_data[1].astype(np.int64)
       
#         return batch_feats, batch_labels

#     def __len__(self):
#         return len(self.spkid2utt)

In [ ]:
# pickle_dataset = PickleDataSet(output)

In [ ]:
# length = len(pickle_dataset)

In [ ]:
# length_list = []

In [ ]:
# for i in range(length):
#     length_list.append(pickle_dataset[i][0].shape[1])
#     if (i+1) % 1000 == 0:
#         print(i+1)

In [ ]:
# len(length_list)

In [ ]:
# for i in length_list:
#     print(i)

# Make validation data (Need only once)

In [30]:
fft = librosa.get_fftlib()
class VoxIterableDataset(object):
    def __init__(self, data_dir_dict, data_len_dict, config):        
        with open(data_dir_dict['spk2utt_train_dict'], 'rb') as handle:
            self.spk2utt_train_dict = pickle.load(handle)
        with open(data_dir_dict['music_dict'], 'rb') as handle:
            self.music_dict = pickle.load(handle)
        with open(data_dir_dict['noise_dict'], 'rb') as handle:
            self.noise_dict = pickle.load(handle)
        with open(data_dir_dict['babble_dict'], 'rb') as handle:
            self.babble_dict = pickle.load(handle)
        with open(data_dir_dict['rir_dict'], 'rb') as handle:
            self.rir_dict = pickle.load(handle)
            
        with open(data_len_dict['spk2utt_train_len'], 'rb') as handle:
            self.spk2utt_train_len = pickle.load(handle)
        with open(data_len_dict['music_len'], 'rb') as handle:
            self.music_len = pickle.load(handle)
        with open(data_len_dict['noise_len'], 'rb') as handle:
            self.noise_len = pickle.load(handle)
        with open(data_len_dict['babble_len'], 'rb') as handle:
            self.babble_len = pickle.load(handle)
        
        
        self.random_spkrs_batchlist = None
        self.ramdom_batch_len = None
        self.random_noise_type = None
        
        
        self.possible_babble_num = [3, 4, 5, 6, 7]
        self.possible_babble_snr = [13, 15, 17, 20]
        self.possible_noise_snr = [0, 5, 10, 15]
        self.possible_music_snr = [5, 8, 10, 15]
        
        self.sr = config['sr']
        self.repeats = config['repeats']
        self.batch_size = config['batch_size']
        self.extended_prefectch = config['extended_prefectch']
        
        self.mfcc_dim = 30
        
        # Auxiliary paras
        self.multi_read_count = 0
        self.preload_mem = False
        
        
        
    def __iter__(self):
        return self
    
    def __next__(self):
        assert len(self.ramdom_batch_len) == len(self.random_spkrs_batchlist)
        try:
            batch_frame_len = self.ramdom_batch_len.pop(0)
            batch_spkrs = self.random_spkrs_batchlist.pop(0)
            batch_noise_type = self.random_noise_type.pop(0)
            batched_feats = np.zeros([self.batch_size, batch_frame_len, self.mfcc_dim])
            batched_labels = np.zeros(self.batch_size)
            
            for batch_index, (spkr, noise_type) in enumerate(zip(batch_spkrs, batch_noise_type)):
                
                concat_wav, VAD_result = self._colleting_and_slicing(spkr, batch_frame_len,\
                hop_len=160, extended_prefectch=self.extended_prefectch)
            
                
                if noise_type == 0:
                    aug_wav = concat_wav
                
                elif noise_type == 1:
                    aug_wav = self._add_rebverb(concat_wav)
                   
                elif noise_type == 2:
                    aug_wav = self._add_noise(concat_wav)
                    
                elif noise_type == 3:
                    aug_wav = self._add_music(concat_wav)
                  
                elif noise_type == 4:
                    aug_wav = self._add_babble(concat_wav)
             
                else:
                    raise NotImplementedError
                    
            
                single_feats = librosa.feature.mfcc(y=aug_wav, sr=self.sr, n_mfcc=30, \
                dct_type=2, n_fft=512, hop_length=160, \
                win_length=None, window='hann', power=2.0, \
                center=True, pad_mode='reflect', n_mels=30, \
                fmin=20, fmax=7600)
                # Note single_feats needs transpose
                out_feats = self._CMVN(single_feats.T, cmn_window = 300, normalize_variance = False)
                # Apply VAD
                assert out_feats.shape[0] == VAD_result.shape[0]
#                 out_feats = out_feats[VAD_result.astype(np.bool)]
                batched_feats[batch_index] = out_feats[:batch_frame_len]
                batched_labels[batch_index] = spkr
                
            return batched_feats, batched_labels
        
        except IndexError:
            raise StopIteration

    def process_one_utt(self, utt_dir):
        try:
            concat_wav, _ = librosa.load(utt_dir, sr=self.sr)
            
            VAD_result = self._VAD_detection(concat_wav)
            
            aug_wav = concat_wav

            single_feats = librosa.feature.mfcc(y=aug_wav, sr=self.sr, n_mfcc=30, \
            dct_type=2, n_fft=512, hop_length=160, \
            win_length=None, window='hann', power=2.0, \
            center=True, pad_mode='reflect', n_mels=30, \
            fmin=20, fmax=7600)
            # Note single_feats needs transpose
            out_feats = self._CMVN(single_feats.T, cmn_window = 300, normalize_variance = False)
            # Apply VAD
            assert out_feats.shape[0] == VAD_result.shape[0]
            out_feats = out_feats[VAD_result.astype(np.bool)]
            
            batched_feats = out_feats[None, :, :]
                
            return batched_feats
        
        except Exception:
            traceback.print_exc()

    def process_one_utt_noVAD(self, utt_dir):
        try:
            concat_wav, _ = librosa.load(utt_dir, sr=self.sr)
            
            aug_wav = concat_wav

            single_feats = librosa.feature.mfcc(y=aug_wav, sr=self.sr, n_mfcc=30, \
            dct_type=2, n_fft=512, hop_length=160, \
            win_length=None, window='hann', power=2.0, \
            center=True, pad_mode='reflect', n_mels=30, \
            fmin=20, fmax=7600)
            # Note single_feats needs transpose
            out_feats = self._CMVN(single_feats.T, cmn_window = 300, normalize_variance = False)
            
            batched_feats = out_feats[None, :, :]
                
            return batched_feats
        
        except Exception:
            traceback.print_exc()            
    
    def noise_data_preload(self):
        print('preloading music_dict')
        for count, i in enumerate(self.music_dict):
            _, _ = librosa.load(self.music_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading noise_dict')        
        for count, i in enumerate(self.noise_dict):
            _, _ = librosa.load(self.noise_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading babble_dict')        
        for count, i in enumerate(self.babble_dict):
            _, _ = librosa.load(self.babble_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
    
    def noise_data_preload2mem(self):
        print('preloading to memory')
        
        self.music_preload_dict = {}
        self.noise_preload_dict = {}
        self.babble_preload_dict = {}
        self.preload_mem = True
        print('preloading music_dict')
        for count, i in enumerate(self.music_dict):
            self.music_preload_dict[i], _ = librosa.load(self.music_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading noise_dict')        
        for count, i in enumerate(self.noise_dict):
            self.noise_preload_dict[i], _ = librosa.load(self.noise_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading babble_dict')        
        for count, i in enumerate(self.babble_dict):
            self.babble_preload_dict[i], _ = librosa.load(self.babble_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)       
        
        
    def get_random_list(self):
        spkrs_list = self.repeats * list(self.spk2utt_train_dict.keys())
        random.shuffle(spkrs_list)
        len_spkrs_list = len(spkrs_list)
        self.random_spkrs_batchlist = [spkrs_list[i*self.batch_size:i*self.batch_size+self.batch_size]\
        for i in range(len_spkrs_list // self.batch_size)]
        
        self.ramdom_batch_len = [random.randint(200, 400) for i in range(len_spkrs_list // self.batch_size)]
        
        noise_type_list = [i%5 for i in range(len_spkrs_list)]

        random.shuffle(noise_type_list)
        self.random_noise_type = [noise_type_list[i*self.batch_size:i*self.batch_size+self.batch_size]\
        for i in range(len_spkrs_list // self.batch_size)]
        
        assert len(self.random_spkrs_batchlist) == len(self.ramdom_batch_len)\
        == len(self.random_noise_type)
        
    def _colleting_and_slicing(self, spkr, batch_frame_len, hop_len=160, extended_prefectch=2.0):
        
        least_wav_len = (batch_frame_len - 1) * hop_len
        concat_utt = np.zeros(0)
        valid_frames_len = 0
        
        # Use to count multi_read_count
        get_count = 0

        while valid_frames_len < batch_frame_len:
#             concat_utt = np.zeros(0)

            utt_dir = self._get_random_spk_utt(spkr, self.spk2utt_train_dict)
            utt_len = self.spk2utt_train_len[utt_dir]
#             off = self._get_random_offset(least_wav_len, utt_len) / self.sr
            off = self._get_random_offset(least_wav_len+extended_prefectch*self.sr, utt_len) / self.sr
            dur = least_wav_len / self.sr + extended_prefectch
            
            utt_part, _ = librosa.load(utt_dir, sr=self.sr, offset=off, duration=dur)
            
            concat_utt = np.append(concat_utt, utt_part)
            detected_frames = self._VAD_detection(concat_utt)
            valid_frames_len = np.sum(detected_frames)

            get_count += 1

        if get_count > 1:
            self.multi_read_count += 1

        VAD_result = detected_frames
        return concat_utt, VAD_result
    
    def _add_rebverb(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = in_wav
        filter_dir = self._get_random_noise(self.rir_dict)
        filter, _ = librosa.load(filter_dir, sr=self.sr)
        
        signal_length = len(signal)
        filter_length = len(filter)
        output_length = signal_length + filter_length - 1
        output = np.zeros(output_length)

        fft_length = 2**np.ceil(np.log2(4 * filter_length)).astype(np.int)
        block_length = fft_length - filter_length + 1


        filter_padded = np.zeros(fft_length)
        filter_padded[0:filter_length] = filter
        filter_padded = fft.rfft(filter_padded)



        for i in range(signal_length//block_length + 1):
            process_length = min(block_length, signal_length - i * block_length);
            signal_block_padded = np.zeros(fft_length)
            signal_block_padded[0:process_length] = signal[i * block_length : i * block_length + process_length]
            signal_block_padded = fft.rfft(signal_block_padded)

            signal_block_padded = filter_padded * signal_block_padded

            signal_block_padded = fft.irfft(signal_block_padded, n=fft_length)

            if (i*block_length + fft_length) <= output_length:
                output[i*block_length : i*block_length + fft_length] += signal_block_padded
            else:
                output[i*block_length : output_length] += signal_block_padded[:output_length-i*block_length]
        
        # shift with max index of filter
        shift_index = np.argmax(filter)
        
        final_out = output[shift_index:shift_index+signal_length]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_noise(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]
        
        signal_len = len(signal)
        total_noise_len = 0
        signal_off = 0
        while total_noise_len < signal_len:
            
            noise_dir, noise_index = self._get_random_noise(self.noise_dict, return_index=True)
            noise_len = self.noise_len[noise_index]
            if noise_len > signal_len:
                noise_off = self._get_random_offset(signal_len, noise_len)
                total_noise_len += signal_len
                if self.preload_mem:
                    noise = self.noise_preload_dict[noise_index][noise_off:noise_off+signal_len]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr, offset=noise_off/self.sr,\
                    duration=signal_len/self.sr)
                
            else:
                total_noise_len += noise_len
                if self.preload_mem:
                    noise = self.noise_preload_dict[noise_index]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr)
                
            snr_db = self.possible_noise_snr[random.randint(0, len(self.possible_noise_snr)-1)]
        
            signal = self._add_db(signal, noise, signal_off, snr_db, power_before_reverb)
            
            signal_off += len(noise)
        
        output = signal
        final_out = output[shift_index:shift_index+signal_len]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_music(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]
        
        signal_len = len(signal)
        total_noise_len = 0
        signal_off = 0
        while total_noise_len < signal_len:
            
            noise_dir, noise_index = self._get_random_noise(self.music_dict, return_index=True)
            noise_len = self.music_len[noise_index]
            if noise_len > signal_len:
                noise_off = self._get_random_offset(signal_len, noise_len)
                total_noise_len += signal_len
                if self.preload_mem:
                    noise = self.music_preload_dict[noise_index][noise_off:noise_off+signal_len]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr, offset=noise_off/self.sr,\
                    duration=signal_len/self.sr)
            else:
                total_noise_len += noise_len
                if self.preload_mem:
                    noise = self.music_preload_dict[noise_index]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr)
                
            snr_db = self.possible_music_snr[random.randint(0, len(self.possible_music_snr)-1)]
        
            signal = self._add_db(signal, noise, signal_off, snr_db, power_before_reverb)
            
            signal_off += len(noise)
        
        output = signal
        final_out = output[shift_index:shift_index+signal_len]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_babble(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]
        
        signal_len = len(signal)
        signal_off = 0
        bg_spks_num = self.possible_babble_num[random.randint(0, len(self.possible_babble_num)-1)]    
        for _ in range(bg_spks_num):            
            noise_dir, noise_index = self._get_random_noise(self.babble_dict, return_index=True)
            noise_len = self.babble_len[noise_index]
            if noise_len > signal_len:
                noise_off = self._get_random_offset(signal_len, noise_len)
                if self.preload_mem:
                    noise = self.babble_preload_dict[noise_index][noise_off:noise_off+signal_len]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr, offset=noise_off/self.sr,\
                    duration=signal_len/self.sr)
            else:
                if self.preload_mem:
                    noise = self.babble_preload_dict[noise_index]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr)
                
            snr_db = self.possible_babble_snr[random.randint(0, len(self.possible_babble_snr)-1)]
        
            signal = self._add_db(signal, noise, signal_off, snr_db, power_before_reverb)
            
        output = signal
        final_out = output[shift_index:shift_index+signal_len]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_db(self, in_wav, noise, signal_off, snr_db, power_before_reverb):
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]

        noise_power = noise.dot(noise) / len(noise)
        scale_factor = np.sqrt(10**(-snr_db / 10) * power_before_reverb / noise_power)
        noise = scale_factor * noise

        add_length = min(len(noise), len(signal)-signal_off)
        signal[signal_off:signal_off+add_length] += noise[:add_length]
        out_wav = signal      
        
        return out_wav
    
    def _CMVN(self, in_feat, cmn_window = 300, normalize_variance = False):             
        num_frames = in_feat.shape[0]
        dim = in_feat.shape[1]
        last_window_start = -1
        last_window_end = -1
        cur_sum = np.zeros(dim)
        cur_sumsq = np.zeros(dim)

        out_feat = np.zeros([num_frames, dim])

        for t in range(num_frames):
            window_start = 0
            window_end = 0

            window_start = t - int(cmn_window / 2)
            window_end = window_start + cmn_window

            if (window_start < 0):
                window_end -= window_start
                window_start = 0

            if (window_end > num_frames):
                window_start -= (window_end - num_frames)
                window_end = num_frames
                if (window_start < 0):
                    window_start = 0

            if (last_window_start == -1):
                input_part = in_feat[window_start:window_end]
                cur_sum = np.sum(input_part, axis=0, keepdims=False)
                if normalize_variance:
                    cur_sumsq = np.sum(input_part**2, axis=0, keepdims=False)
            else:
                if (window_start > last_window_start):
                    frame_to_remove = in_feat[last_window_start]
                    cur_sum -= frame_to_remove
                    if normalize_variance:
                        cur_sumsq -= frame_to_remove**2

                if (window_end > last_window_end):
                    frame_to_add = in_feat[last_window_end]
                    cur_sum += frame_to_add
                    if normalize_variance:
                        cur_sumsq += frame_to_add**2

            window_frames = window_end - window_start
            last_window_start = window_start
            last_window_end = window_end

            out_feat[t] = in_feat[t] - (1.0 / window_frames) * cur_sum


            if normalize_variance:
                if (window_frames == 1):
                    out_feat[t] = 0.0
                else:
                    variance = (1.0 / window_frames) * cur_sumsq - (1.0 / window_frames**2) * cur_sum**2
                    variance = np.maximum(1.0e-10, variance)
                    out_feat[t] /= variance**(0.5)
                    
        return out_feat

    def _get_random_noise(self, noise_dict, return_index=False):
        dict_len = len(noise_dict)
        i = random.randint(0, dict_len-1)
        noise_dir = noise_dict[i]
        
        if return_index:
            return noise_dir, i
        else:
            return noise_dir
    
    def _get_random_spk_utt(self, spkr, spk2utt):
        this_utts = spk2utt[spkr]
        this_num_utts = len(this_utts)
        i = random.randint(0, this_num_utts-1)
        utt_dir = this_utts[i]
        return utt_dir

    def _get_random_offset(self, expected_length, utt_len):
        if expected_length > utt_len:
            return 0
        
        free_length = utt_len - expected_length
        offset = random.randint(0, free_length)
        return offset
        
    @property
    def _VAD_config(self):
        vad_energy_threshold = -3.0
        vad_energy_mean_scale = 1.0
        vad_frames_context = 0
        vad_proportion_threshold = 0.12
        
        return vad_energy_threshold, vad_energy_mean_scale,\
        vad_frames_context, vad_proportion_threshold
        
        
    def _VAD_detection(self, wav):
        vad_energy_threshold, vad_energy_mean_scale,\
        vad_frames_context, vad_proportion_threshold = self._VAD_config
        
        y_tmp = np.pad(wav, int(512 // 2), mode='reflect')
        y_tmp = librosa.util.frame(y_tmp, frame_length=512, hop_length=160)
        y_log_energy = np.log(np.maximum(np.sum(y_tmp**2, axis=0), 1e-15))

        T = len(y_log_energy)
        output_voiced = np.zeros(T)
        if (T == 0):
            raise Exception("zero wave length")

        energy_threshold = vad_energy_threshold
        if (vad_energy_mean_scale != 0.0):
            assert(vad_energy_mean_scale > 0.0)
            energy_threshold += vad_energy_mean_scale * np.sum(y_log_energy) / T


        assert(vad_frames_context >= 0)
        assert(vad_proportion_threshold > 0.0 and vad_proportion_threshold < 1.0);

        for t in range(T):
            num_count = 0
            den_count = 0
            context = vad_frames_context
            for t2 in range(t - context, t + context+1):
                if (t2 >= 0 and t2 < T):
                    den_count+=1
                    if (y_log_energy[t2] > energy_threshold):
                        num_count+=1

            if (num_count >= den_count * vad_proportion_threshold):
                output_voiced[t] = 1.0
            else:
                output_voiced[t] = 0.0
        
        return output_voiced

In [27]:
# OPT_INDEX = '/Lun0/zhiyong/dataset'
data_dir_dict = {}

# val
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_val_dict')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX_1, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX_1, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX_1, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX_1, 'rir_dict')

In [28]:
data_len_dict = {}

data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX, 'spk2utt_val_len')
data_len_dict['music_len'] = os.path.join(OPT_INDEX_1, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX_1, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX_1, 'babble_len')

In [31]:
config = {}

config['sr'] = 16000
config['repeats'] = 5
config['batch_size'] = 1
config['extended_prefectch'] = 1.0

def iter_data_preload(dataset, i):
    dataset.get_random_list()
    try:
        start_time = time.time()
        for count, (data, label) in enumerate(dataset):
            with open('/Lun0/zhiyong/vox_small_noVAD/val_data/'+str(count), 'wb') as handle:
                pickle.dump((data.astype(np.float16), label.astype(np.int16)), handle)
            print(time.time()-start_time)
            with open('/Lun0/zhiyong/vox_small_noVAD/tmp_data_errlog', 'a') as f:
                f.write(str(time.time()-start_time)+'\n')
            start_time = time.time()
    except Exception:
        with open('/Lun0/zhiyong/vox_small_noVAD/tmp_data_errlog', 'a') as f:
            traceback.print_exc(file=f)
        raise Exception

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)
# dataset.noise_data_preload2mem()
dataset.noise_data_preload()

processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(1)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

preloading music_dict
100
200
300
400
500
600
preloading noise_dict
100
200
300
400
500
600
700
800
900
preloading babble_dict
100
200
300
400
0.03362607955932617
0.025624990463256836
0.024207592010498047
0.02437305450439453
0.025974273681640625
0.028914690017700195
0.02845144271850586
0.024558067321777344
0.0176699161529541
0.02493762969970703
0.019082307815551758
0.02550029754638672
0.014556169509887695
0.01964569091796875
0.016458988189697266
0.03236198425292969
0.02210068702697754
0.021233320236206055
0.03284287452697754
0.017510652542114258
0.032269954681396484
0.023085355758666992
0.02485489845275879
0.009671211242675781
0.026090145111083984
0.03376173973083496
0.009717702865600586
0.026825428009033203
0.025084733963012695
0.028535842895507812
0.015878915786743164
0.019161462783813477
0.013719797134399414
0.03726935386657715
0.039730072021484375
0.0229799747467041
0.03039240837097168
0.016979455947875977
0.03183484077453613
0.03339028358459473
0.02453470230102539
0.02965974807739

0.011260271072387695
0.02393031120300293
0.014375448226928711
0.020922183990478516
0.013087749481201172
0.010308265686035156
0.008130073547363281
0.009827375411987305
0.00925898551940918
0.009218931198120117
0.0324854850769043
0.024584054946899414
0.017452239990234375
0.01130533218383789
0.017960548400878906
0.009083747863769531
0.015386581420898438
0.018180131912231445
0.029191970825195312
0.011344671249389648
0.008288145065307617
0.012563228607177734
0.01293492317199707
0.024227619171142578
0.030930042266845703
0.011010408401489258
0.013675451278686523
0.01325368881225586
0.011567115783691406
0.014435052871704102
0.016139745712280273
0.017960786819458008
0.014478445053100586
0.011049032211303711
0.022057056427001953
0.009665489196777344
0.02428746223449707
0.0133514404296875
0.01653885841369629
0.02286672592163086
0.019525527954101562
0.012664079666137695
0.00869297981262207
0.016309499740600586
0.010994672775268555
0.018364906311035156
0.02268695831298828
0.012259721755981445
0.0161

0.01587820053100586
0.015939950942993164
0.061666011810302734
0.012576103210449219
0.008864164352416992
0.07565021514892578
0.012359857559204102
0.01091313362121582
0.011283159255981445
0.023980140686035156
0.011966705322265625
0.013584375381469727
0.03638911247253418
0.010419130325317383
0.011505842208862305
0.010552167892456055
0.012064695358276367
0.020873308181762695
0.008770942687988281
0.013015985488891602
0.01700592041015625
0.011188507080078125
0.010194063186645508
0.014757633209228516
0.014031410217285156
0.014154911041259766
0.011130809783935547
0.013676643371582031
0.014814615249633789
0.02778029441833496
0.01470637321472168
0.01673722267150879
0.0196988582611084
0.01351308822631836
0.012213468551635742
0.009282112121582031
0.016335487365722656
0.026043176651000977
0.011679410934448242
0.010058164596557617
0.011658668518066406
0.01715874671936035
0.010711908340454102
0.014791727066040039
0.015809059143066406
0.013871908187866211
0.010037422180175781
0.013501644134521484
0.01

# Make validation index (Need only once)

In [32]:
# OPT_INDEX = '/Lun0/zhiyong/dataset'
train_data_dir = os.path.join(OPT_INDEX, 'val_data')
expected_len = 1000
workers = 1
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'val_data_csv.csv')

In [33]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [34]:
count = 0
with open(output, 'w') as f:
    for j in range(single_worker_len):
        path = os.path.join(train_data_dir, str(j))
        assert os.path.isfile(path)
        f.write(path+'\n')
        count += 1

In [35]:
count

1000

# Make trial data (Need only once)

In [36]:
fft = librosa.get_fftlib()
class VoxIterableDataset(object):
    def __init__(self, data_dir_dict, data_len_dict, config):        
        with open(data_dir_dict['spk2utt_train_dict'], 'rb') as handle:
            self.spk2utt_train_dict = pickle.load(handle)
        with open(data_dir_dict['music_dict'], 'rb') as handle:
            self.music_dict = pickle.load(handle)
        with open(data_dir_dict['noise_dict'], 'rb') as handle:
            self.noise_dict = pickle.load(handle)
        with open(data_dir_dict['babble_dict'], 'rb') as handle:
            self.babble_dict = pickle.load(handle)
        with open(data_dir_dict['rir_dict'], 'rb') as handle:
            self.rir_dict = pickle.load(handle)
            
        with open(data_len_dict['spk2utt_train_len'], 'rb') as handle:
            self.spk2utt_train_len = pickle.load(handle)
        with open(data_len_dict['music_len'], 'rb') as handle:
            self.music_len = pickle.load(handle)
        with open(data_len_dict['noise_len'], 'rb') as handle:
            self.noise_len = pickle.load(handle)
        with open(data_len_dict['babble_len'], 'rb') as handle:
            self.babble_len = pickle.load(handle)
        
        
        self.random_spkrs_batchlist = None
        self.ramdom_batch_len = None
        self.random_noise_type = None
        
        
        self.possible_babble_num = [3, 4, 5, 6, 7]
        self.possible_babble_snr = [13, 15, 17, 20]
        self.possible_noise_snr = [0, 5, 10, 15]
        self.possible_music_snr = [5, 8, 10, 15]
        
        self.sr = config['sr']
        self.repeats = config['repeats']
        self.batch_size = config['batch_size']
        self.extended_prefectch = config['extended_prefectch']
        
        self.mfcc_dim = 30
        
        # Auxiliary paras
        self.multi_read_count = 0
        self.preload_mem = False
        
        
        
    def __iter__(self):
        return self
    
    def __next__(self):
        assert len(self.ramdom_batch_len) == len(self.random_spkrs_batchlist)
        try:
            batch_frame_len = self.ramdom_batch_len.pop(0)
            batch_spkrs = self.random_spkrs_batchlist.pop(0)
            batch_noise_type = self.random_noise_type.pop(0)
            batched_feats = np.zeros([self.batch_size, batch_frame_len, self.mfcc_dim])
            batched_labels = np.zeros(self.batch_size)
            
            for batch_index, (spkr, noise_type) in enumerate(zip(batch_spkrs, batch_noise_type)):
                
                concat_wav, VAD_result = self._colleting_and_slicing(spkr, batch_frame_len,\
                hop_len=160, extended_prefectch=self.extended_prefectch)
            
                
                if noise_type == 0:
                    aug_wav = concat_wav
                
                elif noise_type == 1:
                    aug_wav = self._add_rebverb(concat_wav)
                   
                elif noise_type == 2:
                    aug_wav = self._add_noise(concat_wav)
                    
                elif noise_type == 3:
                    aug_wav = self._add_music(concat_wav)
                  
                elif noise_type == 4:
                    aug_wav = self._add_babble(concat_wav)
             
                else:
                    raise NotImplementedError
                    
            
                single_feats = librosa.feature.mfcc(y=aug_wav, sr=self.sr, n_mfcc=30, \
                dct_type=2, n_fft=512, hop_length=160, \
                win_length=None, window='hann', power=2.0, \
                center=True, pad_mode='reflect', n_mels=30, \
                fmin=20, fmax=7600)
                # Note single_feats needs transpose
                out_feats = self._CMVN(single_feats.T, cmn_window = 300, normalize_variance = False)
                # Apply VAD
                assert out_feats.shape[0] == VAD_result.shape[0]
#                 out_feats = out_feats[VAD_result.astype(np.bool)]
                batched_feats[batch_index] = out_feats[:batch_frame_len]
                batched_labels[batch_index] = spkr
                
            return batched_feats, batched_labels
        
        except IndexError:
            raise StopIteration

    def process_one_utt(self, utt_dir):
        try:
            concat_wav, _ = librosa.load(utt_dir, sr=self.sr)
            
            VAD_result = self._VAD_detection(concat_wav)
            
            aug_wav = concat_wav

            single_feats = librosa.feature.mfcc(y=aug_wav, sr=self.sr, n_mfcc=30, \
            dct_type=2, n_fft=512, hop_length=160, \
            win_length=None, window='hann', power=2.0, \
            center=True, pad_mode='reflect', n_mels=30, \
            fmin=20, fmax=7600)
            # Note single_feats needs transpose
            out_feats = self._CMVN(single_feats.T, cmn_window = 300, normalize_variance = False)
            # Apply VAD
            assert out_feats.shape[0] == VAD_result.shape[0]
            out_feats = out_feats[VAD_result.astype(np.bool)]
            
            batched_feats = out_feats[None, :, :]
                
            return batched_feats
        
        except Exception:
            traceback.print_exc()

    def process_one_utt_noVAD(self, utt_dir):
        try:
            concat_wav, _ = librosa.load(utt_dir, sr=self.sr)
            
            aug_wav = concat_wav

            single_feats = librosa.feature.mfcc(y=aug_wav, sr=self.sr, n_mfcc=30, \
            dct_type=2, n_fft=512, hop_length=160, \
            win_length=None, window='hann', power=2.0, \
            center=True, pad_mode='reflect', n_mels=30, \
            fmin=20, fmax=7600)
            # Note single_feats needs transpose
            out_feats = self._CMVN(single_feats.T, cmn_window = 300, normalize_variance = False)
            
            batched_feats = out_feats[None, :, :]
                
            return batched_feats
        
        except Exception:
            traceback.print_exc()            
    
    def noise_data_preload(self):
        print('preloading music_dict')
        for count, i in enumerate(self.music_dict):
            _, _ = librosa.load(self.music_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading noise_dict')        
        for count, i in enumerate(self.noise_dict):
            _, _ = librosa.load(self.noise_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading babble_dict')        
        for count, i in enumerate(self.babble_dict):
            _, _ = librosa.load(self.babble_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
    
    def noise_data_preload2mem(self):
        print('preloading to memory')
        
        self.music_preload_dict = {}
        self.noise_preload_dict = {}
        self.babble_preload_dict = {}
        self.preload_mem = True
        print('preloading music_dict')
        for count, i in enumerate(self.music_dict):
            self.music_preload_dict[i], _ = librosa.load(self.music_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading noise_dict')        
        for count, i in enumerate(self.noise_dict):
            self.noise_preload_dict[i], _ = librosa.load(self.noise_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading babble_dict')        
        for count, i in enumerate(self.babble_dict):
            self.babble_preload_dict[i], _ = librosa.load(self.babble_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)       
        
        
    def get_random_list(self):
        spkrs_list = self.repeats * list(self.spk2utt_train_dict.keys())
        random.shuffle(spkrs_list)
        len_spkrs_list = len(spkrs_list)
        self.random_spkrs_batchlist = [spkrs_list[i*self.batch_size:i*self.batch_size+self.batch_size]\
        for i in range(len_spkrs_list // self.batch_size)]
        
        self.ramdom_batch_len = [random.randint(200, 400) for i in range(len_spkrs_list // self.batch_size)]
        
        noise_type_list = [i%5 for i in range(len_spkrs_list)]

        random.shuffle(noise_type_list)
        self.random_noise_type = [noise_type_list[i*self.batch_size:i*self.batch_size+self.batch_size]\
        for i in range(len_spkrs_list // self.batch_size)]
        
        assert len(self.random_spkrs_batchlist) == len(self.ramdom_batch_len)\
        == len(self.random_noise_type)
        
    def _colleting_and_slicing(self, spkr, batch_frame_len, hop_len=160, extended_prefectch=2.0):
        
        least_wav_len = (batch_frame_len - 1) * hop_len
        concat_utt = np.zeros(0)
        valid_frames_len = 0
        
        # Use to count multi_read_count
        get_count = 0

        while valid_frames_len < batch_frame_len:
#             concat_utt = np.zeros(0)

            utt_dir = self._get_random_spk_utt(spkr, self.spk2utt_train_dict)
            utt_len = self.spk2utt_train_len[utt_dir]
#             off = self._get_random_offset(least_wav_len, utt_len) / self.sr
            off = self._get_random_offset(least_wav_len+extended_prefectch*self.sr, utt_len) / self.sr
            dur = least_wav_len / self.sr + extended_prefectch
            
            utt_part, _ = librosa.load(utt_dir, sr=self.sr, offset=off, duration=dur)
            
            concat_utt = np.append(concat_utt, utt_part)
            detected_frames = self._VAD_detection(concat_utt)
            valid_frames_len = np.sum(detected_frames)

            get_count += 1

        if get_count > 1:
            self.multi_read_count += 1

        VAD_result = detected_frames
        return concat_utt, VAD_result
    
    def _add_rebverb(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = in_wav
        filter_dir = self._get_random_noise(self.rir_dict)
        filter, _ = librosa.load(filter_dir, sr=self.sr)
        
        signal_length = len(signal)
        filter_length = len(filter)
        output_length = signal_length + filter_length - 1
        output = np.zeros(output_length)

        fft_length = 2**np.ceil(np.log2(4 * filter_length)).astype(np.int)
        block_length = fft_length - filter_length + 1


        filter_padded = np.zeros(fft_length)
        filter_padded[0:filter_length] = filter
        filter_padded = fft.rfft(filter_padded)



        for i in range(signal_length//block_length + 1):
            process_length = min(block_length, signal_length - i * block_length);
            signal_block_padded = np.zeros(fft_length)
            signal_block_padded[0:process_length] = signal[i * block_length : i * block_length + process_length]
            signal_block_padded = fft.rfft(signal_block_padded)

            signal_block_padded = filter_padded * signal_block_padded

            signal_block_padded = fft.irfft(signal_block_padded, n=fft_length)

            if (i*block_length + fft_length) <= output_length:
                output[i*block_length : i*block_length + fft_length] += signal_block_padded
            else:
                output[i*block_length : output_length] += signal_block_padded[:output_length-i*block_length]
        
        # shift with max index of filter
        shift_index = np.argmax(filter)
        
        final_out = output[shift_index:shift_index+signal_length]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_noise(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]
        
        signal_len = len(signal)
        total_noise_len = 0
        signal_off = 0
        while total_noise_len < signal_len:
            
            noise_dir, noise_index = self._get_random_noise(self.noise_dict, return_index=True)
            noise_len = self.noise_len[noise_index]
            if noise_len > signal_len:
                noise_off = self._get_random_offset(signal_len, noise_len)
                total_noise_len += signal_len
                if self.preload_mem:
                    noise = self.noise_preload_dict[noise_index][noise_off:noise_off+signal_len]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr, offset=noise_off/self.sr,\
                    duration=signal_len/self.sr)
                
            else:
                total_noise_len += noise_len
                if self.preload_mem:
                    noise = self.noise_preload_dict[noise_index]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr)
                
            snr_db = self.possible_noise_snr[random.randint(0, len(self.possible_noise_snr)-1)]
        
            signal = self._add_db(signal, noise, signal_off, snr_db, power_before_reverb)
            
            signal_off += len(noise)
        
        output = signal
        final_out = output[shift_index:shift_index+signal_len]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_music(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]
        
        signal_len = len(signal)
        total_noise_len = 0
        signal_off = 0
        while total_noise_len < signal_len:
            
            noise_dir, noise_index = self._get_random_noise(self.music_dict, return_index=True)
            noise_len = self.music_len[noise_index]
            if noise_len > signal_len:
                noise_off = self._get_random_offset(signal_len, noise_len)
                total_noise_len += signal_len
                if self.preload_mem:
                    noise = self.music_preload_dict[noise_index][noise_off:noise_off+signal_len]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr, offset=noise_off/self.sr,\
                    duration=signal_len/self.sr)
            else:
                total_noise_len += noise_len
                if self.preload_mem:
                    noise = self.music_preload_dict[noise_index]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr)
                
            snr_db = self.possible_music_snr[random.randint(0, len(self.possible_music_snr)-1)]
        
            signal = self._add_db(signal, noise, signal_off, snr_db, power_before_reverb)
            
            signal_off += len(noise)
        
        output = signal
        final_out = output[shift_index:shift_index+signal_len]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_babble(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]
        
        signal_len = len(signal)
        signal_off = 0
        bg_spks_num = self.possible_babble_num[random.randint(0, len(self.possible_babble_num)-1)]    
        for _ in range(bg_spks_num):            
            noise_dir, noise_index = self._get_random_noise(self.babble_dict, return_index=True)
            noise_len = self.babble_len[noise_index]
            if noise_len > signal_len:
                noise_off = self._get_random_offset(signal_len, noise_len)
                if self.preload_mem:
                    noise = self.babble_preload_dict[noise_index][noise_off:noise_off+signal_len]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr, offset=noise_off/self.sr,\
                    duration=signal_len/self.sr)
            else:
                if self.preload_mem:
                    noise = self.babble_preload_dict[noise_index]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr)
                
            snr_db = self.possible_babble_snr[random.randint(0, len(self.possible_babble_snr)-1)]
        
            signal = self._add_db(signal, noise, signal_off, snr_db, power_before_reverb)
            
        output = signal
        final_out = output[shift_index:shift_index+signal_len]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_db(self, in_wav, noise, signal_off, snr_db, power_before_reverb):
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]

        noise_power = noise.dot(noise) / len(noise)
        scale_factor = np.sqrt(10**(-snr_db / 10) * power_before_reverb / noise_power)
        noise = scale_factor * noise

        add_length = min(len(noise), len(signal)-signal_off)
        signal[signal_off:signal_off+add_length] += noise[:add_length]
        out_wav = signal      
        
        return out_wav
    
    def _CMVN(self, in_feat, cmn_window = 300, normalize_variance = False):             
        num_frames = in_feat.shape[0]
        dim = in_feat.shape[1]
        last_window_start = -1
        last_window_end = -1
        cur_sum = np.zeros(dim)
        cur_sumsq = np.zeros(dim)

        out_feat = np.zeros([num_frames, dim])

        for t in range(num_frames):
            window_start = 0
            window_end = 0

            window_start = t - int(cmn_window / 2)
            window_end = window_start + cmn_window

            if (window_start < 0):
                window_end -= window_start
                window_start = 0

            if (window_end > num_frames):
                window_start -= (window_end - num_frames)
                window_end = num_frames
                if (window_start < 0):
                    window_start = 0

            if (last_window_start == -1):
                input_part = in_feat[window_start:window_end]
                cur_sum = np.sum(input_part, axis=0, keepdims=False)
                if normalize_variance:
                    cur_sumsq = np.sum(input_part**2, axis=0, keepdims=False)
            else:
                if (window_start > last_window_start):
                    frame_to_remove = in_feat[last_window_start]
                    cur_sum -= frame_to_remove
                    if normalize_variance:
                        cur_sumsq -= frame_to_remove**2

                if (window_end > last_window_end):
                    frame_to_add = in_feat[last_window_end]
                    cur_sum += frame_to_add
                    if normalize_variance:
                        cur_sumsq += frame_to_add**2

            window_frames = window_end - window_start
            last_window_start = window_start
            last_window_end = window_end

            out_feat[t] = in_feat[t] - (1.0 / window_frames) * cur_sum


            if normalize_variance:
                if (window_frames == 1):
                    out_feat[t] = 0.0
                else:
                    variance = (1.0 / window_frames) * cur_sumsq - (1.0 / window_frames**2) * cur_sum**2
                    variance = np.maximum(1.0e-10, variance)
                    out_feat[t] /= variance**(0.5)
                    
        return out_feat

    def _get_random_noise(self, noise_dict, return_index=False):
        dict_len = len(noise_dict)
        i = random.randint(0, dict_len-1)
        noise_dir = noise_dict[i]
        
        if return_index:
            return noise_dir, i
        else:
            return noise_dir
    
    def _get_random_spk_utt(self, spkr, spk2utt):
        this_utts = spk2utt[spkr]
        this_num_utts = len(this_utts)
        i = random.randint(0, this_num_utts-1)
        utt_dir = this_utts[i]
        return utt_dir

    def _get_random_offset(self, expected_length, utt_len):
        if expected_length > utt_len:
            return 0
        
        free_length = utt_len - expected_length
        offset = random.randint(0, free_length)
        return offset
        
    @property
    def _VAD_config(self):
        vad_energy_threshold = -3.0
        vad_energy_mean_scale = 1.0
        vad_frames_context = 0
        vad_proportion_threshold = 0.12
        
        return vad_energy_threshold, vad_energy_mean_scale,\
        vad_frames_context, vad_proportion_threshold
        
        
    def _VAD_detection(self, wav):
        vad_energy_threshold, vad_energy_mean_scale,\
        vad_frames_context, vad_proportion_threshold = self._VAD_config
        
        y_tmp = np.pad(wav, int(512 // 2), mode='reflect')
        y_tmp = librosa.util.frame(y_tmp, frame_length=512, hop_length=160)
        y_log_energy = np.log(np.maximum(np.sum(y_tmp**2, axis=0), 1e-15))

        T = len(y_log_energy)
        output_voiced = np.zeros(T)
        if (T == 0):
            raise Exception("zero wave length")

        energy_threshold = vad_energy_threshold
        if (vad_energy_mean_scale != 0.0):
            assert(vad_energy_mean_scale > 0.0)
            energy_threshold += vad_energy_mean_scale * np.sum(y_log_energy) / T


        assert(vad_frames_context >= 0)
        assert(vad_proportion_threshold > 0.0 and vad_proportion_threshold < 1.0);

        for t in range(T):
            num_count = 0
            den_count = 0
            context = vad_frames_context
            for t2 in range(t - context, t + context+1):
                if (t2 >= 0 and t2 < T):
                    den_count+=1
                    if (y_log_energy[t2] > energy_threshold):
                        num_count+=1

            if (num_count >= den_count * vad_proportion_threshold):
                output_voiced[t] = 1.0
            else:
                output_voiced[t] = 0.0
        
        return output_voiced

In [37]:
# OPT_INDEX = '/Lun0/zhiyong/dataset'
data_dir_dict = {}

# val
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_val_dict')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX_1, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX_1, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX_1, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX_1, 'rir_dict')

In [38]:
data_len_dict = {}

data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX, 'spk2utt_val_len')
data_len_dict['music_len'] = os.path.join(OPT_INDEX_1, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX_1, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX_1, 'babble_len')

In [40]:
config = {}

config['sr'] = 16000
config['repeats'] = None
config['batch_size'] = None
config['extended_prefectch'] = None

trial_dict_dir = '/Lun0/zhiyong/vox_small_noVAD/trial_dict'

def trial_data_preload(dataset, i, trial_dict_dir):
    with open(trial_dict_dir, 'rb') as handle:
        trial_dict = pickle.load(handle)
        
    start_time = time.time()
    for i in trial_dict:
        data = dataset.process_one_utt_noVAD(trial_dict[i])
        lbpart = trial_dict[i].split('/')[-3:]
        lbpart = lbpart[0]+'-'+lbpart[1]+'-'+lbpart[2]
        label = lbpart[:-4]
        with open('/Lun0/zhiyong/vox_small_noVAD/trial_data/'+str(i), 'wb') as handle:
            pickle.dump((data.astype(np.float16), [label]), handle)
        print(time.time()-start_time)
        start_time = time.time()

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)

processes = [Process(target = trial_data_preload, args = (dataset, i, trial_dict_dir)) for i in range(1)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

0.048090457916259766
0.018772602081298828
0.021146059036254883
0.02582240104675293
0.0190887451171875
0.008682727813720703
0.017663240432739258
0.013514995574951172
0.027914047241210938
0.011623382568359375
0.024919509887695312
0.03120255470275879
0.009030818939208984
0.01413583755493164
0.021210432052612305
0.009571075439453125
0.022357702255249023
0.018395423889160156
0.011065483093261719
0.016025781631469727
0.01649618148803711
0.009105443954467773
0.03390789031982422
0.016454696655273438
0.029098033905029297
0.011958122253417969
0.01969146728515625
0.019387483596801758
0.06854867935180664
0.016950368881225586
0.01625680923461914
0.028841257095336914
0.025653362274169922
0.010080099105834961
0.00988626480102539
0.044972896575927734
0.009733200073242188
0.014881372451782227
0.01335906982421875
0.01604318618774414
0.008692264556884766
0.010566234588623047
0.015877723693847656
0.010752439498901367
0.011837005615234375
0.009948968887329102
0.01639533042907715
0.010280370712280273
0.0103

0.011139154434204102
0.015908479690551758
0.00988149642944336
0.014261245727539062
0.015035390853881836
0.011345386505126953
0.011117219924926758
0.012256860733032227
0.01665472984313965
0.010444164276123047
0.014900922775268555
0.01286005973815918
0.009545564651489258
0.010781049728393555
0.012961149215698242
0.00922846794128418
0.010241031646728516
0.023743391036987305
0.009227514266967773
0.014130830764770508
0.013177156448364258
0.012178182601928711
0.01629781723022461
0.009493112564086914
0.010028600692749023
0.019193172454833984
0.011108636856079102
0.014762401580810547
0.02476024627685547
0.013588428497314453
0.009015083312988281
0.015499114990234375
0.008743762969970703
0.01363992691040039
0.013251781463623047
0.011433124542236328
0.008661985397338867
0.019891977310180664
0.009298324584960938
0.012220382690429688
0.016947269439697266
0.012632608413696289
0.02476978302001953
0.008554220199584961
0.021933317184448242
0.012823343276977539
0.02867293357849121
0.015229463577270508
0

0.012680768966674805
0.014110326766967773
0.026771068572998047
0.011570930480957031
0.00991368293762207
0.02297663688659668
0.020943641662597656
0.011882781982421875
0.008971452713012695
0.010474205017089844
0.16049504280090332
0.008578300476074219
0.010968446731567383
0.012208223342895508
0.03458237648010254
0.05047440528869629
0.009183883666992188
0.010466575622558594
0.011252403259277344
0.08560490608215332
0.04249310493469238
0.022087574005126953
0.07781004905700684
0.015569925308227539
0.01409912109375
0.021468639373779297
0.04630470275878906
0.01651310920715332
0.012655496597290039
0.0431671142578125
0.01481175422668457
0.018871784210205078
0.02521800994873047
0.022991180419921875
0.009473323822021484
0.009190559387207031
0.013560771942138672
0.010365486145019531
0.027405500411987305
0.01564621925354004
0.014134407043457031
0.015067100524902344
0.01560211181640625
0.008836507797241211
0.02589702606201172
0.031150341033935547
0.01623058319091797
0.026193857192993164
0.009520053863

0.02280735969543457
0.010663032531738281
0.01243734359741211
0.010199785232543945
0.009132623672485352
0.04337263107299805
0.035021066665649414
0.026027917861938477
0.010220766067504883
0.0186307430267334
0.008676290512084961
0.008470773696899414
0.015920639038085938
0.014627933502197266
0.025079965591430664
0.010865449905395508
0.011080503463745117
0.012011528015136719
0.0159451961517334
0.010409116744995117
0.019962549209594727
0.018097400665283203
0.01031351089477539
0.01694178581237793
0.014311552047729492
0.009163379669189453
0.011209726333618164
0.015344619750976562
0.009227275848388672
0.018790483474731445
0.01599597930908203
0.01250910758972168
0.02171802520751953
0.010346651077270508
0.014703750610351562
0.014608621597290039
0.031656742095947266
0.010034799575805664
0.00874781608581543
0.014523029327392578
0.013222932815551758
0.023109912872314453
0.024171829223632812
0.05753016471862793
0.019980907440185547
0.02693939208984375
0.019446134567260742
0.016652584075927734
0.07648

0.043821096420288086
0.051513671875
0.017512798309326172
0.01102900505065918
0.045902252197265625
0.022825956344604492
0.011549711227416992
0.025413036346435547
0.03198075294494629
0.04470396041870117
0.03411602973937988
0.017439603805541992
0.01143193244934082
0.03493547439575195
0.008533954620361328
0.04011726379394531
0.015076875686645508
0.030326128005981445
0.010890960693359375
0.008387327194213867
0.007313966751098633
0.011250972747802734
0.014338970184326172
0.018909215927124023
0.009105682373046875
0.020178556442260742
0.008280754089355469
0.013089418411254883
0.012439250946044922
0.009379148483276367
0.008301258087158203
0.01570296287536621
0.015488386154174805
0.023653745651245117
0.00900125503540039
0.011533737182617188
0.008192777633666992
0.008807659149169922
0.01197052001953125
0.007596492767333984
0.007683515548706055
0.008930444717407227
0.0064127445220947266
0.01287841796875
0.008048534393310547
0.008258581161499023
0.007878303527832031
0.012495994567871094
0.008627176

...

0.010735034942626953
0.010549545288085938
0.01301121711730957
0.011442899703979492
0.009425163269042969
0.00899195671081543
0.00908207893371582
0.013624429702758789
0.010837793350219727
0.008658885955810547
0.010268926620483398
0.013251304626464844
0.008644580841064453
0.010024785995483398
0.010203361511230469
0.014344930648803711
0.02626490592956543
0.00962066650390625
0.009365558624267578
0.017450332641601562
0.03460812568664551
0.046506404876708984
0.021495580673217773
0.016282081604003906
0.016420364379882812
0.028092384338378906
0.01616811752319336
0.016863107681274414
0.01970839500427246
0.02188849449157715
0.019650697708129883
0.018339872360229492
0.01291966438293457
0.014772415161132812
0.03048992156982422
0.012111186981201172
0.022297382354736328
0.027935266494750977
0.016414880752563477
0.020602703094482422
0.008543014526367188
0.031717777252197266
0.024853944778442383
0.011502981185913086
0.04295468330383301
0.009848594665527344
0.008968353271484375
0.2816808223724365
0.0342

0.01926255226135254
0.01319742202758789
0.00863337516784668
0.011588335037231445
0.012992382049560547
0.0088958740234375
0.011145353317260742
0.031116247177124023
0.015975236892700195
0.01202249526977539
0.02743244171142578
0.008827924728393555
0.012582778930664062
0.009704351425170898
0.009922266006469727
0.009891033172607422
0.01222538948059082
0.024756431579589844
0.009433507919311523
0.018680810928344727
0.030059099197387695
0.014507532119750977
0.009260177612304688
0.007547616958618164
0.019884347915649414
0.009840726852416992
0.00926971435546875
0.006453990936279297
0.009538650512695312
0.02245640754699707
0.07829928398132324
0.015504598617553711
0.006877422332763672
0.010673046112060547
0.007643222808837891
0.015602350234985352
0.007679462432861328
0.0074307918548583984
0.010985851287841797
0.01523137092590332
0.007548332214355469
0.019304752349853516
0.024796724319458008
0.01870250701904297
0.021974802017211914
0.009058475494384766
0.0229799747467041
0.008461952209472656
0.0065

0.007951974868774414
0.006825923919677734
0.00623774528503418
0.005675315856933594
0.00881338119506836
0.005761146545410156
0.008711576461791992
0.009765863418579102
0.005609989166259766
0.010885238647460938
0.0061948299407958984
0.006327152252197266
0.014778614044189453
0.005882978439331055
0.007450103759765625
0.0057888031005859375
0.00582122802734375
0.0055005550384521484
0.010027170181274414
0.022165298461914062
0.008603811264038086
0.00556182861328125
0.009051322937011719
0.007153749465942383
0.011597871780395508
0.007914543151855469
0.006941556930541992
0.00623631477355957
0.005436420440673828
0.00942230224609375
0.008819341659545898
0.0054819583892822266
0.009283304214477539
0.005649089813232422
0.008978128433227539
0.02775096893310547
0.015401840209960938
0.015246152877807617
0.010584831237792969
0.016340255737304688
0.028206348419189453
0.010326147079467773
0.036536455154418945
0.03401041030883789
0.015151023864746094
0.011780738830566406
0.010198354721069336
0.008066892623901

0.02109360694885254
0.016573429107666016
0.010163545608520508
0.010043144226074219
0.011098623275756836
0.11307144165039062
0.019481182098388672
0.03954052925109863
0.041559696197509766
0.013654232025146484
0.03474068641662598
0.011943578720092773
0.01347041130065918
0.02977895736694336
0.044625282287597656
0.02104020118713379
0.011505842208862305
0.023053407669067383
0.018410444259643555
0.01238560676574707
0.04119157791137695
0.015262365341186523
0.01136636734008789
0.018979549407958984
0.01064610481262207
0.024889707565307617
0.027590036392211914
0.012302637100219727
0.009318828582763672
0.010804176330566406
0.011335611343383789
0.010827064514160156
0.011127948760986328
0.011470794677734375
0.012482404708862305
0.015913009643554688
0.013659000396728516
0.01641678810119629
0.03647494316101074
0.015486001968383789
0.013158321380615234
0.014617919921875
0.029807567596435547
0.012347221374511719
0.01736140251159668
0.011061429977416992
0.026376724243164062
0.017231225967407227
0.0300841

0.012185096740722656
0.030905961990356445
0.009498357772827148
0.026365995407104492
0.010748863220214844
0.024264097213745117
0.009456872940063477
0.008550405502319336
0.01079702377319336
0.021555662155151367
0.018255949020385742
0.019599199295043945
0.02262401580810547
0.009647369384765625
0.022686481475830078
0.03943467140197754
0.008902549743652344
0.011183738708496094
0.017914533615112305
0.009557008743286133
0.019472122192382812
0.02872490882873535
0.008548974990844727
0.019983291625976562
0.008428573608398438
0.010734796524047852
0.008220434188842773
0.008914470672607422
0.008926630020141602
0.012744665145874023
0.016013383865356445
0.0339047908782959
0.01929783821105957
0.008504629135131836
0.013160943984985352
0.018172025680541992
0.019744157791137695
0.013665437698364258
0.010567903518676758
0.019409894943237305
0.009978771209716797
0.026198863983154297
0.009642839431762695
0.016150712966918945
0.009315252304077148
0.016041994094848633
0.012229204177856445
0.008525609970092773

# Make trial index (Need only once)

In [41]:
# OPT_INDEX = '/Lun0/zhiyong/dataset'
train_data_dir = os.path.join(OPT_INDEX, 'trial_data')
expected_len = 4874
workers = 1
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'trial_data_csv.csv')

In [42]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [43]:
count = 0
with open(output, 'w') as f:
    for j in range(single_worker_len):
        path = os.path.join(train_data_dir, str(j))
        assert os.path.isfile(path)
        f.write(path+'\n')
        count += 1

In [44]:
count

4874

# Make PLDA

In [46]:
fft = librosa.get_fftlib()
class VoxIterableDataset(object):
    def __init__(self, data_dir_dict, data_len_dict, config):        
        with open(data_dir_dict['spk2utt_train_dict'], 'rb') as handle:
            self.spk2utt_train_dict = pickle.load(handle)
        with open(data_dir_dict['music_dict'], 'rb') as handle:
            self.music_dict = pickle.load(handle)
        with open(data_dir_dict['noise_dict'], 'rb') as handle:
            self.noise_dict = pickle.load(handle)
        with open(data_dir_dict['babble_dict'], 'rb') as handle:
            self.babble_dict = pickle.load(handle)
        with open(data_dir_dict['rir_dict'], 'rb') as handle:
            self.rir_dict = pickle.load(handle)
            
        with open(data_len_dict['spk2utt_train_len'], 'rb') as handle:
            self.spk2utt_train_len = pickle.load(handle)
        with open(data_len_dict['music_len'], 'rb') as handle:
            self.music_len = pickle.load(handle)
        with open(data_len_dict['noise_len'], 'rb') as handle:
            self.noise_len = pickle.load(handle)
        with open(data_len_dict['babble_len'], 'rb') as handle:
            self.babble_len = pickle.load(handle)
        
        
        self.random_spkrs_batchlist = None
        self.ramdom_batch_len = None
        self.random_noise_type = None
        
        
        self.possible_babble_num = [3, 4, 5, 6, 7]
        self.possible_babble_snr = [13, 15, 17, 20]
        self.possible_noise_snr = [0, 5, 10, 15]
        self.possible_music_snr = [5, 8, 10, 15]
        
        self.sr = config['sr']
        self.repeats = config['repeats']
        self.batch_size = config['batch_size']
        self.extended_prefectch = config['extended_prefectch']
        
        self.mfcc_dim = 30
        
        # Auxiliary paras
        self.multi_read_count = 0
        self.preload_mem = False
        
        
        
    def __iter__(self):
        return self
    
    def __next__(self):
        assert len(self.ramdom_batch_len) == len(self.random_spkrs_batchlist)
        try:
            batch_frame_len = self.ramdom_batch_len.pop(0)
            batch_spkrs = self.random_spkrs_batchlist.pop(0)
            batch_noise_type = self.random_noise_type.pop(0)
            batched_feats = np.zeros([self.batch_size, batch_frame_len, self.mfcc_dim])
            batched_labels = np.zeros(self.batch_size)
            
            for batch_index, (spkr, noise_type) in enumerate(zip(batch_spkrs, batch_noise_type)):
                
                concat_wav, VAD_result = self._colleting_and_slicing(spkr, batch_frame_len,\
                hop_len=160, extended_prefectch=self.extended_prefectch)
            
                
                if noise_type == 0:
                    aug_wav = concat_wav
                
                elif noise_type == 1:
                    aug_wav = self._add_rebverb(concat_wav)
                   
                elif noise_type == 2:
                    aug_wav = self._add_noise(concat_wav)
                    
                elif noise_type == 3:
                    aug_wav = self._add_music(concat_wav)
                  
                elif noise_type == 4:
                    aug_wav = self._add_babble(concat_wav)
             
                else:
                    raise NotImplementedError
                    
            
                single_feats = librosa.feature.mfcc(y=aug_wav, sr=self.sr, n_mfcc=30, \
                dct_type=2, n_fft=512, hop_length=160, \
                win_length=None, window='hann', power=2.0, \
                center=True, pad_mode='reflect', n_mels=30, \
                fmin=20, fmax=7600)
                # Note single_feats needs transpose
                out_feats = self._CMVN(single_feats.T, cmn_window = 300, normalize_variance = False)
                # Apply VAD
                assert out_feats.shape[0] == VAD_result.shape[0]
                out_feats = out_feats[VAD_result.astype(np.bool)]
                batched_feats[batch_index] = out_feats[:batch_frame_len]
                batched_labels[batch_index] = spkr
                
            return batched_feats, batched_labels
        
        except IndexError:
            raise StopIteration

    def process_one_utt(self, utt_dir):
        try:
            concat_wav, _ = librosa.load(utt_dir, sr=self.sr)
            
            VAD_result = self._VAD_detection(concat_wav)
            
            aug_wav = concat_wav

            single_feats = librosa.feature.mfcc(y=aug_wav, sr=self.sr, n_mfcc=30, \
            dct_type=2, n_fft=512, hop_length=160, \
            win_length=None, window='hann', power=2.0, \
            center=True, pad_mode='reflect', n_mels=30, \
            fmin=20, fmax=7600)
            # Note single_feats needs transpose
            out_feats = self._CMVN(single_feats.T, cmn_window = 300, normalize_variance = False)
            # Apply VAD
            assert out_feats.shape[0] == VAD_result.shape[0]
            out_feats = out_feats[VAD_result.astype(np.bool)]
            
            batched_feats = out_feats[None, :, :]
                
            return batched_feats
        
        except Exception:
            traceback.print_exc()

    def process_one_utt_noVAD(self, utt_dir):
        try:
            concat_wav, _ = librosa.load(utt_dir, sr=self.sr)
            
            aug_wav = concat_wav

            single_feats = librosa.feature.mfcc(y=aug_wav, sr=self.sr, n_mfcc=30, \
            dct_type=2, n_fft=512, hop_length=160, \
            win_length=None, window='hann', power=2.0, \
            center=True, pad_mode='reflect', n_mels=30, \
            fmin=20, fmax=7600)
            # Note single_feats needs transpose
            out_feats = self._CMVN(single_feats.T, cmn_window = 300, normalize_variance = False)
            
            batched_feats = out_feats[None, :, :]
                
            return batched_feats
        
        except Exception:
            traceback.print_exc()            
    
    def noise_data_preload(self):
        print('preloading music_dict')
        for count, i in enumerate(self.music_dict):
            _, _ = librosa.load(self.music_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading noise_dict')        
        for count, i in enumerate(self.noise_dict):
            _, _ = librosa.load(self.noise_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading babble_dict')        
        for count, i in enumerate(self.babble_dict):
            _, _ = librosa.load(self.babble_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
    
    def noise_data_preload2mem(self):
        print('preloading to memory')
        
        self.music_preload_dict = {}
        self.noise_preload_dict = {}
        self.babble_preload_dict = {}
        self.preload_mem = True
        print('preloading music_dict')
        for count, i in enumerate(self.music_dict):
            self.music_preload_dict[i], _ = librosa.load(self.music_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading noise_dict')        
        for count, i in enumerate(self.noise_dict):
            self.noise_preload_dict[i], _ = librosa.load(self.noise_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading babble_dict')        
        for count, i in enumerate(self.babble_dict):
            self.babble_preload_dict[i], _ = librosa.load(self.babble_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)       
        
        
    def get_random_list(self):
        spkrs_list = self.repeats * list(self.spk2utt_train_dict.keys())
        random.shuffle(spkrs_list)
        len_spkrs_list = len(spkrs_list)
        self.random_spkrs_batchlist = [spkrs_list[i*self.batch_size:i*self.batch_size+self.batch_size]\
        for i in range(len_spkrs_list // self.batch_size)]
        
        self.ramdom_batch_len = [random.randint(200, 400) for i in range(len_spkrs_list // self.batch_size)]
        
        noise_type_list = [i%5 for i in range(len_spkrs_list)]

        random.shuffle(noise_type_list)
        self.random_noise_type = [noise_type_list[i*self.batch_size:i*self.batch_size+self.batch_size]\
        for i in range(len_spkrs_list // self.batch_size)]
        
        assert len(self.random_spkrs_batchlist) == len(self.ramdom_batch_len)\
        == len(self.random_noise_type)
        
    def _colleting_and_slicing(self, spkr, batch_frame_len, hop_len=160, extended_prefectch=2.0):
        
        least_wav_len = (batch_frame_len - 1) * hop_len
        concat_utt = np.zeros(0)
        valid_frames_len = 0
        
        # Use to count multi_read_count
        get_count = 0

        while valid_frames_len < batch_frame_len:
            concat_utt = np.zeros(0)

            utt_dir = self._get_random_spk_utt(spkr, self.spk2utt_train_dict)
            utt_len = self.spk2utt_train_len[utt_dir]
#             off = self._get_random_offset(least_wav_len, utt_len) / self.sr
            off = self._get_random_offset(least_wav_len+extended_prefectch*self.sr, utt_len) / self.sr
            dur = least_wav_len / self.sr + extended_prefectch
            
            utt_part, _ = librosa.load(utt_dir, sr=self.sr, offset=off, duration=dur)
            
            concat_utt = np.append(concat_utt, utt_part)
            detected_frames = self._VAD_detection(concat_utt)
            valid_frames_len = np.sum(detected_frames)

            get_count += 1

        if get_count > 1:
            self.multi_read_count += 1

        VAD_result = detected_frames
        return concat_utt, VAD_result
    
    def _add_rebverb(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = in_wav
        filter_dir = self._get_random_noise(self.rir_dict)
        filter, _ = librosa.load(filter_dir, sr=self.sr)
        
        signal_length = len(signal)
        filter_length = len(filter)
        output_length = signal_length + filter_length - 1
        output = np.zeros(output_length)

        fft_length = 2**np.ceil(np.log2(4 * filter_length)).astype(np.int)
        block_length = fft_length - filter_length + 1


        filter_padded = np.zeros(fft_length)
        filter_padded[0:filter_length] = filter
        filter_padded = fft.rfft(filter_padded)



        for i in range(signal_length//block_length + 1):
            process_length = min(block_length, signal_length - i * block_length);
            signal_block_padded = np.zeros(fft_length)
            signal_block_padded[0:process_length] = signal[i * block_length : i * block_length + process_length]
            signal_block_padded = fft.rfft(signal_block_padded)

            signal_block_padded = filter_padded * signal_block_padded

            signal_block_padded = fft.irfft(signal_block_padded, n=fft_length)

            if (i*block_length + fft_length) <= output_length:
                output[i*block_length : i*block_length + fft_length] += signal_block_padded
            else:
                output[i*block_length : output_length] += signal_block_padded[:output_length-i*block_length]
        
        # shift with max index of filter
        shift_index = np.argmax(filter)
        
        final_out = output[shift_index:shift_index+signal_length]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_noise(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]
        
        signal_len = len(signal)
        total_noise_len = 0
        signal_off = 0
        while total_noise_len < signal_len:
            
            noise_dir, noise_index = self._get_random_noise(self.noise_dict, return_index=True)
            noise_len = self.noise_len[noise_index]
            if noise_len > signal_len:
                noise_off = self._get_random_offset(signal_len, noise_len)
                total_noise_len += signal_len
                if self.preload_mem:
                    noise = self.noise_preload_dict[noise_index][noise_off:noise_off+signal_len]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr, offset=noise_off/self.sr,\
                    duration=signal_len/self.sr)
                
            else:
                total_noise_len += noise_len
                if self.preload_mem:
                    noise = self.noise_preload_dict[noise_index]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr)
                
            snr_db = self.possible_noise_snr[random.randint(0, len(self.possible_noise_snr)-1)]
        
            signal = self._add_db(signal, noise, signal_off, snr_db, power_before_reverb)
            
            signal_off += len(noise)
        
        output = signal
        final_out = output[shift_index:shift_index+signal_len]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_music(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]
        
        signal_len = len(signal)
        total_noise_len = 0
        signal_off = 0
        while total_noise_len < signal_len:
            
            noise_dir, noise_index = self._get_random_noise(self.music_dict, return_index=True)
            noise_len = self.music_len[noise_index]
            if noise_len > signal_len:
                noise_off = self._get_random_offset(signal_len, noise_len)
                total_noise_len += signal_len
                if self.preload_mem:
                    noise = self.music_preload_dict[noise_index][noise_off:noise_off+signal_len]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr, offset=noise_off/self.sr,\
                    duration=signal_len/self.sr)
            else:
                total_noise_len += noise_len
                if self.preload_mem:
                    noise = self.music_preload_dict[noise_index]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr)
                
            snr_db = self.possible_music_snr[random.randint(0, len(self.possible_music_snr)-1)]
        
            signal = self._add_db(signal, noise, signal_off, snr_db, power_before_reverb)
            
            signal_off += len(noise)
        
        output = signal
        final_out = output[shift_index:shift_index+signal_len]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_babble(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]
        
        signal_len = len(signal)
        signal_off = 0
        bg_spks_num = self.possible_babble_num[random.randint(0, len(self.possible_babble_num)-1)]    
        for _ in range(bg_spks_num):            
            noise_dir, noise_index = self._get_random_noise(self.babble_dict, return_index=True)
            noise_len = self.babble_len[noise_index]
            if noise_len > signal_len:
                noise_off = self._get_random_offset(signal_len, noise_len)
                if self.preload_mem:
                    noise = self.babble_preload_dict[noise_index][noise_off:noise_off+signal_len]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr, offset=noise_off/self.sr,\
                    duration=signal_len/self.sr)
            else:
                if self.preload_mem:
                    noise = self.babble_preload_dict[noise_index]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr)
                
            snr_db = self.possible_babble_snr[random.randint(0, len(self.possible_babble_snr)-1)]
        
            signal = self._add_db(signal, noise, signal_off, snr_db, power_before_reverb)
            
        output = signal
        final_out = output[shift_index:shift_index+signal_len]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_db(self, in_wav, noise, signal_off, snr_db, power_before_reverb):
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]

        noise_power = noise.dot(noise) / len(noise)
        scale_factor = np.sqrt(10**(-snr_db / 10) * power_before_reverb / noise_power)
        noise = scale_factor * noise

        add_length = min(len(noise), len(signal)-signal_off)
        signal[signal_off:signal_off+add_length] += noise[:add_length]
        out_wav = signal      
        
        return out_wav
    
    def _CMVN(self, in_feat, cmn_window = 300, normalize_variance = False):             
        num_frames = in_feat.shape[0]
        dim = in_feat.shape[1]
        last_window_start = -1
        last_window_end = -1
        cur_sum = np.zeros(dim)
        cur_sumsq = np.zeros(dim)

        out_feat = np.zeros([num_frames, dim])

        for t in range(num_frames):
            window_start = 0
            window_end = 0

            window_start = t - int(cmn_window / 2)
            window_end = window_start + cmn_window

            if (window_start < 0):
                window_end -= window_start
                window_start = 0

            if (window_end > num_frames):
                window_start -= (window_end - num_frames)
                window_end = num_frames
                if (window_start < 0):
                    window_start = 0

            if (last_window_start == -1):
                input_part = in_feat[window_start:window_end]
                cur_sum = np.sum(input_part, axis=0, keepdims=False)
                if normalize_variance:
                    cur_sumsq = np.sum(input_part**2, axis=0, keepdims=False)
            else:
                if (window_start > last_window_start):
                    frame_to_remove = in_feat[last_window_start]
                    cur_sum -= frame_to_remove
                    if normalize_variance:
                        cur_sumsq -= frame_to_remove**2

                if (window_end > last_window_end):
                    frame_to_add = in_feat[last_window_end]
                    cur_sum += frame_to_add
                    if normalize_variance:
                        cur_sumsq += frame_to_add**2

            window_frames = window_end - window_start
            last_window_start = window_start
            last_window_end = window_end

            out_feat[t] = in_feat[t] - (1.0 / window_frames) * cur_sum


            if normalize_variance:
                if (window_frames == 1):
                    out_feat[t] = 0.0
                else:
                    variance = (1.0 / window_frames) * cur_sumsq - (1.0 / window_frames**2) * cur_sum**2
                    variance = np.maximum(1.0e-10, variance)
                    out_feat[t] /= variance**(0.5)
                    
        return out_feat

    def _get_random_noise(self, noise_dict, return_index=False):
        dict_len = len(noise_dict)
        i = random.randint(0, dict_len-1)
        noise_dir = noise_dict[i]
        
        if return_index:
            return noise_dir, i
        else:
            return noise_dir
    
    def _get_random_spk_utt(self, spkr, spk2utt):
        this_utts = spk2utt[spkr]
        this_num_utts = len(this_utts)
        i = random.randint(0, this_num_utts-1)
        utt_dir = this_utts[i]
        return utt_dir

    def _get_random_offset(self, expected_length, utt_len):
        if expected_length > utt_len:
            return 0
        
        free_length = utt_len - expected_length
        offset = random.randint(0, free_length)
        return offset
        
    @property
    def _VAD_config(self):
        vad_energy_threshold = -3.0
        vad_energy_mean_scale = 1.0
        vad_frames_context = 0
        vad_proportion_threshold = 0.12
        
        return vad_energy_threshold, vad_energy_mean_scale,\
        vad_frames_context, vad_proportion_threshold
        
        
    def _VAD_detection(self, wav):
        vad_energy_threshold, vad_energy_mean_scale,\
        vad_frames_context, vad_proportion_threshold = self._VAD_config
        
        y_tmp = np.pad(wav, int(512 // 2), mode='reflect')
        y_tmp = librosa.util.frame(y_tmp, frame_length=512, hop_length=160)
        y_log_energy = np.log(np.maximum(np.sum(y_tmp**2, axis=0), 1e-15))

        T = len(y_log_energy)
        output_voiced = np.zeros(T)
        if (T == 0):
            raise Exception("zero wave length")

        energy_threshold = vad_energy_threshold
        if (vad_energy_mean_scale != 0.0):
            assert(vad_energy_mean_scale > 0.0)
            energy_threshold += vad_energy_mean_scale * np.sum(y_log_energy) / T


        assert(vad_frames_context >= 0)
        assert(vad_proportion_threshold > 0.0 and vad_proportion_threshold < 1.0);

        for t in range(T):
            num_count = 0
            den_count = 0
            context = vad_frames_context
            for t2 in range(t - context, t + context+1):
                if (t2 >= 0 and t2 < T):
                    den_count+=1
                    if (y_log_energy[t2] > energy_threshold):
                        num_count+=1

            if (num_count >= den_count * vad_proportion_threshold):
                output_voiced[t] = 1.0
            else:
                output_voiced[t] = 0.0
        
        return output_voiced

In [47]:
OPT_INDEX = '/Lun0/zhiyong/dataset'
data_dir_dict = {}

# val
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_val_dict')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX, 'rir_dict')

In [48]:
data_len_dict = {}

data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX, 'spk2utt_val_len')
data_len_dict['music_len'] = os.path.join(OPT_INDEX, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX, 'babble_len')

In [49]:
spk2utt_dict = os.path.join(OPT_INDEX, 'spk2utt_dict')

with open(spk2utt_dict, 'rb') as handle:
    spk2utt_dict_data = pickle.load(handle)
count = 0
for i in spk2utt_dict_data:
    count += len(spk2utt_dict_data[i])

In [50]:
count

1276888

In [53]:
config = {}

config['sr'] = 16000
config['repeats'] = None
config['batch_size'] = None
config['extended_prefectch'] = None

# trial_dict_dir = '/Lun0/zhiyong/dataset/trial_dict'

def trial_data_preload(dataset, i, spk2utt_dict_data):
        
    for i in spk2utt_dict_data:
        for j, line in enumerate(spk2utt_dict_data[i]):
#             data = dataset.process_one_utt(line)
            data = dataset.process_one_utt_noVAD(line)
            lbpart = line.split('/')[-3:]
            lbpart = lbpart[0]+'-'+lbpart[1]+'-'+lbpart[2]
            label = lbpart[:-4]
            with open('/Lun0/zhiyong/dataset/plda_full_data_noVAD/'+str(i)+'_'+str(j), 'wb') as handle:
                pickle.dump((data.astype(np.float16), [label]), handle)
        if ((i+1) % 100) == 0:    
            print(i+1)

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)

processes = [Process(target = trial_data_preload, args = (dataset, i, spk2utt_dict_data)) for i in range(1)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
22095.06617307663


# Make PLDA index

In [54]:
OPT_INDEX = '/Lun0/zhiyong/dataset'
train_data_dir = os.path.join(OPT_INDEX, 'plda_full_data_noVAD')
expected_len = 1276888
workers = 1
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'plda_full_data_noVAD.csv')

In [55]:
spk2utt_dict = os.path.join(OPT_INDEX, 'spk2utt_dict')

with open(spk2utt_dict, 'rb') as handle:
    spk2utt_dict_data = pickle.load(handle)

In [56]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [57]:
count = 0
with open(output, 'w') as f:
    for i in spk2utt_dict_data:
        for j in range(len(spk2utt_dict_data[i])):
            path = os.path.join(train_data_dir, str(i)+'_'+str(j))
            assert os.path.isfile(path)
            f.write(path+'\n')
            count += 1

In [58]:
count

1276888